# 1.0 Loading Packages and Importing Data

In [ ]:
# This code allows or prompts you to allow Collab to access your Drive

from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [ ]:
# Import packages
import pandas as pd
import numpy as np
import pandas as pd
import gdown

In [ ]:
# Read the data into our pd
spreadsheet1 = gc.open('PROMPTS_Dat')

worksheet1 = spreadsheet1.worksheet('PROMPTS')

In [ ]:
# The get_all values function gives a list of rows.
rows = worksheet1.get_all_values()

# Then now we convert to a DataFrame and render
df = pd.DataFrame.from_records(rows[1:],columns=rows[0])

In [ ]:
# Read the data into our pd
spreadsheet1 = gc.open('PROMPTS_HF')

worksheet1 = spreadsheet1.worksheet('HF')

In [ ]:
# The get_all values function gives a list of rows.
rows = worksheet1.get_all_values()

# Then now we convert to a DataFrame and render
df_Facilities = pd.DataFrame.from_records(rows[1:],columns=rows[0])

In [ ]:
df_Facilities.head(3)

,Facility_Name
0,Matete Health Center
1,St Ann Hospital
2,Makueni County Referral Hospital


In [ ]:
# Drop rows with all missing values (NaNs)
df_without_empty_rows = df.dropna(axis=0, how='all')


In [ ]:
# Drop columns with all missing values (NaNs)
df_without_empty_columns = df.dropna(axis=1, how='all')


In [ ]:
# drop rows with null values in the 'Invoice ID' column
df_cleaned = df.dropna(subset=['Timestamp'])


In [ ]:
# Confim now number of rows & columns after dropping
total_rows = df_cleaned.shape[0]
print("Total number of rows:", total_rows)

total_columns = df_cleaned.shape[1]
print("Total number of columns:", total_columns)

Total number of rows: 1316
Total number of columns: 46


# 2.0 Data Cleaning

## 2.1 Null Values

In [ ]:
# Checking for total number of null values in the dataset
# so basically isnull function checks all null values while the sum() functions
# sums the total null values for each variable in the dataset.

df_cleaned.isnull().sum()

Timestamp                                                                                                                                                                                                                                                                                                   0
Name of the Program Associate                                                                                                                                                                                                                                                                               0
Date of the facility visit                                                                                                                                                                                                                                                                                  0
Name of the County                                                                            

## 2.2 Typos and Errors

In [ ]:
# The first step with typos is to run frequency and check if there are
# any element that are not needed for analysis
# Since we will analyze 'Name of the Sub county' lets check if it contains any typo

print(df_cleaned['Name of the Sub county'].value_counts())

                               1121
Kangema                           7
Kigumo                            5
Kilifi North                      4
Kirwara                           4
                               ... 
Kiharu sub county                 1
Sigowet/soin                      1
Awendo                            1
Ruaraka/Roysambu sub-county       1
Murang'a  South                   1
Name: Name of the Sub county, Length: 106, dtype: int64


In [ ]:
print(df_cleaned['During this facility visit, which session did you attend?'].value_counts())

CME /Meeting with staff(Nurses)                  804
Meeting with Nurses and health talk with Mums    488
Health talk with mums                             23
                                                   1
Name: During this facility visit, which session did you attend?, dtype: int64


In [ ]:
# Correct wrong entries
# In the code below we replace wrong entries with the correct entries

df_cleaned['During this facility visit, which session did you attend?'] = df_cleaned['During this facility visit, which session did you attend?'].replace({"CME /Meeting with staff(Nurses)": "Meeting with staff(Nurses)",
                                                                 'Meeting with Nurses and health talk with Mums': 'Meeting with nurses and health talk with mums'})


# Now we run again the frequency to observe if the wrong entry is still present

print(df_cleaned['During this facility visit, which session did you attend?'].value_counts())

Meeting with staff(Nurses)                       804
Meeting with nurses and health talk with mums    488
Health talk with mums                             23
                                                   1
Name: During this facility visit, which session did you attend?, dtype: int64


In [ ]:
# Question on number of staff met

# First I renamed the column
df_modified = df_cleaned.rename(columns={
    "For the CME/meeting with staff, how many people were you able to reach": "For the meeting with staff, how many people were you able to reach"
})

# Then converted the values to string to avoid attribute error - since I will want the output in numerical and not decimal places

# Convert the column to strings first - to avoid getting attribute error.
df_modified['For the meeting with staff, how many people were you able to reach'] = df_modified['For the meeting with staff, how many people were you able to reach'].astype(str)


# Replace values in the modified column (cleaning whitespaces and handling typos/variants)

df_modified['For the meeting with staff, how many people were you able to reach'] = df_modified['For the meeting with staff, how many people were you able to reach'].str.strip().replace({
    '5 and 2 support staff': '7',
    '2, 1 student nurse':'3',
    '2 and 4 student nurses':'6',
    '3 nurses and 12 student nurses':'15',
    '3 nurses, 2 student nurses and 1 linda mama staff':'6',
    '2 nurses and 1 support staff':'3',
    '5 and 2 student nurses':'7',
    '1 nurse and 1 nurse aid':'2',
    ' 5 student nurses':'5',
    '5 student nurses':'5',
    '3 nurses, 2 mentor mothers and 10 student nurses':'15',
    '5 nurses, 2 nutritionists and 5 student nurses':'12',
    '1 nurse, 1 support staff':'2',
    '4 MCH 2 at maternity':'4',
    '3(many of the healthcare providers had gone home since we went to the facility abit late)':'3',
    "4 nurses 2 CO 's":'6',
    "1 nursing officer in-charge (the facility has 3 staff; 1 on duty, 1 on leave and 1 on training)":'4',
    '2 nurses':'2',
    '6 Nurses, incharge and a CO':'7',
    'I didnt attend a   CME':'NA',
    '4 nurses and 8 student nurses':'12',
    '4 nurses':'4',
    '3. Facility incharge,MCH incharge and Maternity nurse.':'3',
    "13 ( 8 at MCH and 5 at maternity)":'13',
    "3. The facility in-charge, The Clinical Officer and the contact person(Nurse)":'3',
    '4 nurses and 3 student nurses':'7',
    '2 nurses and 1 support staff':'3',
    '2, 1 student nurse':'3',
    '4 nurses and 5 support staff':'9',
    'N/A':'NA',
    '5 and 2 support staff':'7',
    '1 nurse, 1 support staff':'2',
    '4 nurses':'4',
    '6 Nurses, incharge and a CO':'7',
    '4 nurses and 3 student nurses':'7',
    'N_A':'NA',
    'N/a':'NA'

})

# Convert to numerical values - since the answers were coming out in text
df_modified['For the meeting with staff, how many people were you able to reach'] = pd.to_numeric(df_modified['For the meeting with staff, how many people were you able to reach'], errors='coerce')

# Convert to integer type - I did not want the values in decimal places
df_modified['For the meeting with staff, how many people were you able to reach'] = df_modified['For the meeting with staff, how many people were you able to reach'].astype('Int64')


# Print the value counts of the modified column
print(df_modified['For the meeting with staff, how many people were you able to reach'].value_counts())


2     279
3     275
4     186
5     101
1      86
6      66
7      50
8      30
9      14
12     12
10     10
14      7
15      5
13      5
17      4
19      3
16      3
11      3
23      3
20      2
27      2
28      2
18      2
25      1
21      1
32      1
0       1
36      1
35      1
26      1
Name: For the meeting with staff, how many people were you able to reach, dtype: Int64


In [ ]:
# Question on Types of topics learnt by staff

import pandas as pd


# List of unique responses
response_names = ['About PROMPTS and its Benefits', 'The role of Health Care workers on PROMPTS',
                  'How to increase Active consenting', 'Facility Prompts action plan',
                  'The Enrollment Options', 'Other']

# Fill NaN values with an empty string to avoid getting a type error because of the blank values in the column
df_modified['Which topics  did you discuss with the Nurses/facility staff'].fillna('', inplace=True)

# Create separate columns for each unique response and indicate their presence with 1 or 0
for response in response_names:
    df_modified[response] = df_modified['Which topics  did you discuss with the Nurses/facility staff'].apply(lambda x: 1 if response in x else 0)

#When you apply this lambda function using .apply(), it will transform each element in the Series
# or column into either a 1 or a 0, depending on whether the value of response is present in that element.

# Get the index of the 'Which topics did you discuss with the Nurses/facility staff' column
index_of_main_column = df_modified.columns.get_loc('Which topics  did you discuss with the Nurses/facility staff')

# Reorder columns to place new columns just after 'Which topics did you discuss with the Nurses/facility staff'
for response in response_names:
    index_of_main_column += 1
    new_column = df_modified.pop(response)
    if isinstance(new_column, pd.DataFrame):
        new_column = new_column.iloc[:, 0]  # Take the first column if multiple columns are returned

            # Rename columns based on condition
    if response == "About PROMPTS and its Benefits":
        new_response_name = "PROMPTS and its Benefits"
    elif response == "Other":
        new_response_name = "Other-topic of discussion with nurses"
    else:
        new_response_name = response

    df_modified.insert(index_of_main_column, new_response_name, new_column)




In [ ]:
# Mums reached for health talk

# Converted the column to strings first
df_modified['For the health talk, how many mums were you able to reach'] = df_modified['For the health talk, how many mums were you able to reach'].astype(str)

# Then I replaced the values to clean data
df_modified['For the health talk, how many mums were you able to reach'] = df_modified['For the health talk, how many mums were you able to reach'].str.strip().replace({
        'Not applicable':'NA',
        'N/A':'NA',
        'N/a':'NA',
        'i wasnt  able to join the health talk':'NA',
        'No':'NA',
        '18 Mums 5 PNC 13 ANC':'18',
        'None':'0',
        'none':'0',
        'n/a':'NA',
        'x':'0',
        'NQ':'NA',
        'O':'0',
        '30+':'30',
        'MA':'NA',
        'Na':'NA',
        'X':'0'
})

# Convert to numerical values
df_modified['For the health talk, how many mums were you able to reach'] = pd.to_numeric(df_modified['For the health talk, how many mums were you able to reach'], errors='coerce')

# Convert to integer type - I did not want the values in decimal places
df_modified['For the health talk, how many mums were you able to reach'] = df_modified['For the health talk, how many mums were you able to reach'].astype('Int64')

print(df_modified['For the health talk, how many mums were you able to reach'].value_counts())


0      229
12      58
3       53
5       48
4       41
2       39
10      39
8       28
6       24
1       21
15      20
7       17
20      13
13      11
30      11
14       9
18       9
22       6
25       6
16       5
23       5
17       5
11       5
9        4
24       4
34       3
28       3
27       3
26       2
37       2
46       2
35       2
45       2
57       2
19       2
167      1
39       1
51       1
56       1
48       1
42       1
33       1
40       1
38       1
36       1
47       1
54       1
32       1
Name: For the health talk, how many mums were you able to reach, dtype: Int64


In [ ]:
# What topics did you discuss with the mums during the health talk?

# List of unique responses
response_names = ['What prompts platform is', 'Benefits of prompts',
                  'How to enroll to prompts', 'How to actively consent or opt out of prompts',
                  'How to ask questions', 'Self enrollment']

# Filled NaN values with an empty string to avoid getting a type error because of the blank values in the column
df_modified['What topics did you discuss with the mums during the health talk?'].fillna('', inplace=True)

# Created separate columns for each unique response and indicate their presence with 1 or 0
for response in response_names:
    df_modified[response] = df_modified['What topics did you discuss with the mums during the health talk?'].apply(lambda x: 1 if response in x else 0)

# Created the "Topics_Any other specify" column to hold the text values not picked in the other columns
df_modified['Topics_Any other specify'] = df_modified['What topics did you discuss with the mums during the health talk?'].apply(lambda x: x if all(response not in x for response in response_names) else '')

# Get the index of the 'What topics did you discuss with the mums during the health talk?' column
index_of_main_column = df_modified.columns.get_loc('What topics did you discuss with the mums during the health talk?')

# Reorder columns to place new columns just after 'What topics did you discuss with the mums during the health talk?'
for response in response_names + ['Topics_Any other specify']:
    index_of_main_column += 1
    new_column = df_modified.pop(response)
    df_modified.insert(index_of_main_column, response, new_column)

# Remove any leading or trailing whitespaces from the "Any other specify" column
df_modified['Any other specify'] = df_modified['Topics_Any other specify'].str.strip()

print(df_modified.columns)



Index(['Timestamp', 'Name of the Program Associate',
       'Date of the facility visit', 'Name of the County',
       'Name of the Sub county',
       'During this facility visit, which session did you attend?',
       'For the meeting with staff, how many people were you able to reach',
       'Which topics  did you discuss with the Nurses/facility staff',
       'PROMPTS and its Benefits',
       'The role of Health Care workers on PROMPTS',
       'How to increase Active consenting', 'Facility Prompts action plan',
       'The Enrollment Options', 'Other-topic of discussion with nurses',
       'For the health talk, how many mums were you able to reach',
       'What topics did you discuss with the mums during the health talk?',
       'What prompts platform is', 'Benefits of prompts',
       'How to enroll to prompts',
       'How to actively consent or opt out of prompts', 'How to ask questions',
       'Self enrollment', 'Topics_Any other specify',
       'What are some of the c

In [ ]:
# If you answered Yes above, who is the person speaking to mums about consent sheets?\n(Cadre)


df_modified["If you answered Yes above, who is the person speaking to mums about consent sheets?\n(Cadre)"] = df_modified["If you answered Yes above, who is the person speaking to mums about consent sheets?\n(Cadre)"].str.strip().replace({
    'Nurse':'Nurse',
    'Nurses':'Nurse',
    'Nurses ':'Nurse',
    'the contact persons':'Contact Person',
    'MCH and deputy maternity incharge':'MCH Incharge',
    'Mch incharge':'MCH Incharge',
    'ANC and PNC nurses':'Nurse',
    'The contact persons at the facility and other nurses':'Nurse',
    'ANC nurse and the maternity nurse':'Nurse',
    'The contact persons':'Contact Person',
    'MCH incharge':'MCH Incharge',
    'Incharge and ANC nurse':'Nurse',
    "Nurse in-charge (MCH)":'Nurse',
    'Nurses/Mentor mother':'Nurse',
    'Nurse/mentor mother':'Nurse',
    'Nurse facility incharge and other casual staff':'Nurse',
    'the contact person':'Contact Person',
    'Mentor mother, social woker and ANC nurse':'Mentor Mother',
    'nursing':'Nurse',
    'Incharge she is responsible  for ANC mothers':'MCH Incharge',
    'Maternity Nurse and facility incharge':'Nurse',
    'nurses and mentor mother':'Nurse',
    'nurses':'Nurse',
    'Nurse and Linda Mama officer':'Nurse',
    'the contact person and the other nurses':'Nurse',
    'Deputy MCH incharge':'MCH Incharge',
    'Staff at MCH':'MCH Incharge',
    'Nurse,mentor mother.':'Nurse',
    'the contact persons at the facility':'Contact Person',
    'Mentor mother and data clerk.':'Mentor Mother',
    'Nurse working at MCH':'Nurse',
    'ANC and maternity staff':'MCH Incharge',
    'This is to kick off immediately by health care providers in the mch and maternity.':'MCH Incharge',
    'the contact persons at maternity and ANC':'Contact Person',
    'Nurse at MCH':'Nurse',
    'the nurses in the facility':'Nurse',
    'MCH in charge and a nurse working at MCH':'MCH Incharge',
    'the contacts and nurses at the facility':'Contact Person',
    'the contact person at the facility':'Contact Person',
    "Nurses(MCH),Mentor mother(maternity)":'Mentor Mother',
    'ANC nurse':'Nurse',
    'MCH nurse':'Nurse',
    'Not available at the moment':'NA',
    'The mch and maternity nurses':'Nurse',
    'contact persons ':'Contact Person',
    'contact persons':'Contact Person',
    'Nurse and CO':'Nurse',
    'Nurse and Linda mama agent':'Nurse',
    'Anc nurse ':'Nurse',
    'Facility Nurse':'Nurse',
    'This is to start effectively from when the facility have transitioned':'NA',
    'The maternity and Mch incharge':'MCH Incharge',
    'Mch nurse and Maternity Nurse':'Nurse',
    'Nurses and CHWs.':'Nurse',
    'Anc nurse':'Nurse',
    'Nurses at MCH':'Nurse',
    'MCH nurses':'Nurse',
    'This is to start tomorrow by the mch and maternity staff':'MCH Incharge',
    'MCH and Maternity in-charge':'MCH Incharge',
    'The facility in-charge':'Contact Person',
    'Staff at MCH and post natal ward':'MCH Incharge',
    'The contact persons in the facility':'Contact Person',
    'nurse':'Nurse',
    'nurse ':'Nurse',
    'MCH staff':'MCH Incharge',
    'Nursing Service Manager/ MCH in-charge':'MCH Incharge',
    'This is to be started by 14th/02/2023':'NA',
    'Facility Nurse MCH incharge and cwc nurse':'Nurse',
    'Mch nurse incharge ':'Nurse',
    'Mch nurse incharge':'Nurse',
    'Nurses/mentor mother':'Mentor Mother',
    'This is to be initiated with the consent transition':'NA',
    'Nurse in-charge':'Nurse',
    'ANC nurses ':'Nurse',
    'ANC nurses':'Nurse',
    'Staff nurses in the mch':'Nurse',
    'This is to start immediately by all healthcare providers in the facility':'NA',
    'mentor mothers':'Mentor Mother',
    'nurses/mentor mother.':'Mentor Mother',
    'N/a':'NA',
    'Nurses,mentor mother.':'Mentor Mother',
    'The nurses in the MCH':'Nurse',
    'Mch and maternity incharge':'MCH Incharge',
    'Nurses,mentor mothers':'Mentor Mother',
    'contact persons at the ANC and maternity':'Contact Person',
    'PNC nutritionist, and MCH nurses':'Nurse',
    'the nurses both at ANC and maternity':'Nurse',
    'the conyact persons':'Contact Person',
    'The contacts persons':'Contact Person',
    'Facility in charge ( MCH in-charge has been transferred)':'MCH Incharge',
    'NSM at the maternity, MCH in-charge':'MCH Incharge',
    'Mch in-charge':'MCH Incharge',
    'No incharge and MCH nurse':'Nurse',
    'Nurses/Mentor mothers':'Mentor Mother',
    'Facility in-charge and MCH staff':'MCH Incharge',
    'Healthcare workees in the MCH department.':'MCH Incharge',
    'Healthcare workees in the MCH department.':'MCH Incharge',
    'Nursing cadre':'Nurse',
    'Nurses mch and oparanya care staff':'Nurse',
    'nurses/clinical officer':'Nurse',
    'AnC nurse':'Nurse',
    'This is to be officialised from 8/2/2023':'NA',
    'Linda Mama officer':'Linda Mama Officer',
    'Nurse officer at MCH':'Nurse',
    'Nurse/ mentor mother':'Mentor Mother',
    'This is start immediately with the MCH incharge and the maternity incharge':'MCH Incharge',
    'NSM, MCH and maternity staff':'MCH Incharge',
    'Anc nurse and Maternity nurse':'Nurse',
    'MCH nurse who is also incharge of the GANC':'Nurse',
    'Nurse, linda mama staff':'Nurse',
    'MCH nurse and the NO':'Nurse',
    'MCH Nurse and staff. And maternity Nurse':'Nurse',
    'This is to kick off immediately':'NA',
    'ANC nurse.':'Nurse',
    'Nurse & Mentor mothers':'Nurse',
    'Mch nurse and the mentor mother':'Nurse',
    'contact persons at ANC and maternity':'Contact Person',
    'ANC and Maternity nurse':'Nurse',
    'Not apllicable at the moment of visit.':'NA',
    'N.a':'NA',
    'The incharge  and mentor father':'MCH Incharge',
    'NSM, MCH and Maternity incharge':'MCH Incharge',
    'Not observed at the moment but healthcare providers in the MCH  and maternity will take the matle moving forward.':'NA',
    'The MCH nurse and NO incharge':'Nurse',
    'Hrio and nutritionist and nurse':'HRIO',
    'MCH and Maternity incharges':'MCH Incharge',
    'Staffs in the MCH inclusive of mentor mothers and community health workers.':'Mentor Mother',
    'This is to commence with the inception of the consent sheets by the mch and maternity nurses.':'NA',
    'The nurses in the mch,the maternity side has a challenge on who to talk to mothers.':'Nurse',
    'Mch Nurse and maternity Nurse incharge':'Nurse',
    'Nurses in the MCH effective tomorrow':'Nurse',
    'Mat nurse and MCH nurse':'Nurse',
    'MCH nurse incharge and maternity Nurse.':'Nurse',
    'This is to start immediately with MCH nurses.':'Nurse',
    'ANc nurses':'Nurse',
    'MCH staff and the staff collecting NHIF details':'MCH Incharge',
    'The incharge':'MCH Incharge',
    'Facility incharge and other nurses':'Nurse',
    'N/A- just transitioned to use of consent sheets':'NA',
    'Students(nurses)':'Nurse',
    'Both the contact persons at ANC and maternity':'Contact Person',
    'This is to start immediately after the consent sheets are distributed to the facility':'NA',
    'MCH and maternity staff spearheaded by MCH incharge':'MCH Incharge',
    'This will kick of from Monday 30th,there were no mums when we visited.':'NA',
    'nurses and a mentor mother':'Nurse',
    'The incharge , mentor mother and the contact person':'Mentor Mother',
    'The contact person together with the other staff':'Contact Person',
    'Nursing officer incharge and the MCH incharge':'MCH Incharge',
    'Facility in charge and MCH icharge':'MCH Incharge',
    'Nurses/CO':'Nurse',
    'This is to commence immediately':'NA',
    'MCH nurses and cwc':'Nurse',
    'Acting MCH in-charge':'MCH Incharge',
    '1. The MCH and maternity staff. 2. MCH in-charge':'MCH Incharge',
    'CHV and facility incharge and nurse':'CHV',
    'MCH and cwc nurses':'Nurse',
    'HRIO and MCH nurses.':'HRIO',
    'The nurses at the facility and the contact person':'Nurse',
    'The incharge and ANC nurse':'Nurse',
    'Nurse interns':'Nurse',
    'Mach nurse':'Nurse',
    'MCH staff and the in-charge':'MCH Incharge',
    'Nurses  and HRIOs':'HRIO',
    'ANC- Nurse Faith Twili is responsible for educating mothers on PROMPTS. For PNC, nurse Samuel is the responsible person to ensure mothers get to be educated on PROMPTS':'Nurse',
    'ANC nurse/promotion officer':'Nurse',
    'Nurse in MCH':'Nurse',
    'MCH incharge and nurse at MCH':'Nurse',
    'To start Immediately':'NA',
    'Jane-nurse at the facility':'Nurse',
    'Incharges':'MCH Incharge',
    'Facility incharge (Nurse)':'Nurse',
    'The nursing officer who is also the contact person for the facility.':'Nurse',
    'Nurse and nutritionist.':'Nurse',
    'HRIO,  Nutritionist and MCH/maternity Nurse':'Nurse',
    'The nursing officer incharge':'Nurse',
    'Nurse, hrio, hts':'Nurse',
    'MCH nurse and cwc nurse and Maternity Nurse':'Nurse',
    'Facility HRIO':'HRIO',
    'MCH nurse incharge and Hrio':'Nurse',
    'Nursing Officer incharge':'Nurse',
    'None. It is the first time I am introducing consent sheet transition':'NA',
    'Nurse facility incharge or the nutritionist':'Nurse',
    'MCH staff (nurses) and maternity Nurse':'Nurse',
    'For now there was no discussions about consent sheets.':'NA',
    'NSM(GaNC group focal person)':'Contact Person',
    'Nurses/CHWs':'Nurse',
    'Nurse in the MCH.':'Nurse',
    'Mentor mother and the MCH nurses':'Mentor Mother',
    'Mch and maternity staff':'MCH Incharge',
    'From the MCH side the nurses and students are doing a great job in sensitizing mothers.':'Nurse',
    'MCH and Facility incharge':'MCH Incharge',
    'MCH incharge and the nurse conducting GANC':'MCH Incharge',
    'Nurses/CHV':'Nurse',
    'All the staff because mums are attended to individually and have multiple consultation rooms':'MCH Incharge',
    'The facility incharge':'Contact Person',
    'Nutritionist and triage nurse':'Nurse',
    'Mentor monther':'Mentor Mother',
    'MCH nurses and maternity nurse incharge':'Nurse',
    'MCH Nurse and maternity Nurses':'Nurse',
    'Mch incharge and Mch Nurses and maternity Nurse':'Nurse',
    'Facility mch nurse and Maternity nurses':'Nurse',
    'MCH nurse and Maternity Nurse and that facility NO incharge':'Nurse',
    'The mch nurse who is giving out the ANC services':'Nurse',
    'ANC and Maternity  nurse':'Nurse',
    'Facility MCH nurse and maternity Nurse':'Nurse',
    'The MCH staff and a boresha afya ya mama staff that works in the facility.':'Contact Person',
    'MCH staff. Rotation is done on a monthly basis':'MCH Incharge',
    'Mentor mothers':'Mentor Mother',
    'Nurse and Ipsos staff.':'Nurse',
    'Nutritionist and Hrio':'HRIO',
    'From my observation, I did not see anyone talking to mothers about consenting, though i saw the consent sheets filled up.':'NA',
    'MCH Nurse incharge':'Nurse',
    'The facility was using opt out method but will start engaging mums through consent sheets.':'NA',
    'Nurse (ANC) and Linda mama officer (maternity)':'Nurse',
    'The identified person, mentor mother has not been enrolling mums since when they received the consent forms.':'NA',
    'We didnt have a chance to give to the nurse to explain i launch the program with nurses and mothers':'NA',
    'Nurse and Epsos staff':'Nurse',
    'ANC nurse and Linda mama officer for maternity':'Nurse',
    'The service providers':'Contact Person',
    'MCH and maternity in-charges and staff':'MCH Incharge',
    'Initially the MCH nurse was the one speaking to mums about consent,she went on leave,a new nurse has been recruited during the time of the visit.':'Nurse',
    'Mentor mothers and Nurses':'Mentor Mother',
    'The nurses at the facility':'Nurse',
    'The nurses on duty educate the mothers during the health talks':'Nurse',
    'Facility nurses and student from MCH and maternity.':'Nurse',
    'MCH incharge also incharge of GANC':'MCH Incharge',
    'This is start from 26th Jan by the MCH nurse incharge':'NA',
    'nurse in the MCH':'Nurse',
    'ANC and maternity  nurses':'Nurse',
    'MCH in-charge and nurse':'MCH Incharge',
    'Mch nurse and the NSM':'Nurse',
    'MCH/Maternity in-charge and the contact persons':'Contact Person',
    'Nursing Service Manager':'Nurse',
    'MCH incharge nurse':'Nurse',
    'the contact persons and nurses on duty':'Contact Person',
    'Nursing':'Nurse',
    'Nurse at the MCH and maternity':'Nurse',
    'Mums who only do not know how to read are the ones being sensitized.':'NA',
    'Nurses and Health records':'Nurse',
    'The nurses on duty and the casual':'Nurse',
    'Nurse supported by a CHV':'Nurse',
    'Not applicable for now':'NA',
    'MCH Incharge.':'MCH Incharge',
    'ChV':'CHV',
    'Facility incharge.':'MCH Incharge',
    'NSM and nurse at ANC':'Nurse',
    'nurses in the mch and maternity':'Nurse',
    'incharge ans she also the anc nurse':'Nurse',
    'Nurses in the mch':'Nurse',
    'Facility Nurse, facility incharge and CHV':'Nurse',
    'MCH nurse and facility incharge':'Nurse',
    'Nurse(MCH)':'Nurse',
    'KRCHN and nurses in the mch and maternity':'Nurse',
    'Nurse at ANC and maternity':'Nurse',
    'Facility incharge':'MCH Incharge',
    'Nurse incharge':'Nurse',
    'Nurses in the mch and maternity.':'Nurse',
    'MCH nurse and Maternity Nurse':'Nurse',
    'Nurse and mentor mother':'Mentor Mother',
    'Linda mama officer':'Linda Mama Officer',
    'the contact persons at ANC and maternity':'Contact Person',
    'the contacts persons':'Contact Person',
    'Nurses in the MCH and maternity.':'Nurse',
    "Nurse's":'Nurse',
    'Facility Nurse and HRIO':'Nurse',
    'Nurses and mentor mother':'Mentor Mother',
    'MCH nurse and cwc nurse':'Nurse',
    'anc nurse':'Nurse',
    'anc NURSE':'Nurse',
    'N/A':'NA',
    'Hrio':'HRIO',
    'Nurses.':'Nurse',
    'MCH nurse incharge':'Nurse',
    'The MCH nurse':'Nurse',
    'Mch and maternity Nurse':'Nurse',
    'MCH nurses and maternity Nurse':'Nurse',
    'NSM':'Nurse',
    'NURSES':'Nurse',
    'MCH nurses and maternity nurses':'Nurse',
    'Mch incharge.':'MCH Incharge',
    'contact person at the facility':'Contact Person',
    'Facility in-charge':'MCH Incharge',
    'Nurses and mentor mother.':'Mentor Mother',
    'MCH nurse and maternity Nurse':'Nurse',
    'ANC incharge':'Nurse',
    'MCH Nurse':'Nurse',
    'CHVs':'CHV',
    'COVID casual staff.':'Contact Person',
    'Facility Nurse and nutritionist':'Nurse',
    'At the ANC the nurses on duty educate mothers about PROMPTS during the health talks. At maternity the nurses together with the recordscontact person are doing so.':'Nurse',
    'Clinical officer.':'Clinical Officer',
    'Facility incharge nurse, MCH nurse, nutritionist':'Nurse',
    'ANC Nurse':'Nurse',
    'mentor mother':'Mentor Mother',
    'Incharge':'MCH Incharge',
    'MCH in-charge':'MCH Incharge',
    'Facility nurse incharge':'Nurse',
    'ANC NURSE':'Nurse',
    'ANc nurse':'Nurse',
    'Facility incharge Nurse':'Nurse',
    'MCH and maternity Nurse':'Nurse',
    'MCH in charge':'MCH Incharge',
    'ANC Nurse':'Nurse',
    'The contact persons at the facility':'Contact Person',
    'Mentor mother':'Mentor Mother'


})

print(df_modified["If you answered Yes above, who is the person speaking to mums about consent sheets?\n(Cadre)"].value_counts())


Nurse                 781
                      196
Contact Person        112
MCH Incharge           93
NA                     71
Mentor Mother          31
CHV                     9
HRIO                    7
n/a                     6
None                    5
Linda Mama Officer      4
Clinical Officer        1
Name: If you answered Yes above, who is the person speaking to mums about consent sheets?\n(Cadre), dtype: int64


In [ ]:
# Who is the custodian of the consent sheet at the facility?(Cadre)

df_modified["Who is the custodian of the consent sheet at the facility?(Cadre)"] = df_modified["Who is the custodian of the consent sheet at the facility?(Cadre)"].str.strip().replace({
    'Nurse':'Nurse',
    'Nurse ':'Nurse',
    'Nurses':'Nurse',
    'Maternity incharge,mch incharge':'Nurse in-charge',
    'the contact persons':'Contact Person',
    'Nurses  ':'Nurse',
    'Mch and maternity incharges':'MCH in-charge',
    'The C/O who is the facility incharge.':'Clinical Officer',
    'Mch incharge':'MCH in-charge',
    'incharges':'Nurse in-charge',
    'ANC nurse':'Nurse in-charge',
    'Nursing Officer':'Nurse',
    'contact person-nurse':'Nurse in-charge',
    'Nurse and facility incharge':'Nurse in-charge',
    'Nurse in-charge (MCH)':'Nurse in-charge',
    'contact persons at ANC and maternity':'Contact Person',
    'Facility incharge  Nurse/ mentor':'Nurse in-charge',
    'Facility Nursing Officer incharge':'Nurse in-charge',
    'Facility incharge (CO)':'Clinical Officer',
    "Facility's Deputy in charge":'Nurse in-charge',
    "Deputy MCH incharge":'MCH in-charge',
    'MCH incharge':'MCH in-charge',
    'NSM':'Nurse in-charge',
    'Facility HRIO':'HRIO',
    'MCH incharge and maternity deputy incharge':'MCH in-charge',
    'Deputy nurae incharge':'Nurse in-charge',
    'Facility assistant in-charge':'Nurse in-charge',
    'Facility incharge ( Nurse)':'Nurse in-charge',
    'Nurses/Mentor mother':'Mentor Mother',
    'Facility incharge, maternity Nurse and MCH nurse':'Nurse in-charge',
    'NO incharge':'None',
    'Facility in charge':'Nurse in-charge',
    'MCH and Postnatal in-charges respectively':'Nurse in-charge',
    'Anc incharge':'Nurse in-charge',
    'Respective departmental in-charges ( McH and maternity)':'Nurse in-charge',
    'MCH In charge':'MCH in-charge',
    'Facility Incharge - Nurse':'Nurse in-charge',
    'Facility incharge ':'Nurse in-charge',
    'MCH in-charge for ANC and Linda mama officer for Maternity':'MCH in-charge',
    'Mentor Mother':'Mentor Mother',
    'Data clerk/mentor mother.':'Mentor Mother',
    'MCH incharge(Nurse)Mentor mother(Mat)':'Mentor Mother',
    'Facility incharge':'Nurse in-charge',
    'Nurse contact persons':'Nurse',
    'Mch and mat incharge':'MCH in-charge',
    'Mch Nurse':'Nurse',
    'Mch incharge, antenatal incharge, postnatal incharge':'MCH in-charge',
    'The contact persons at the facility':'Contact Person',
    'MCH inCharge':'MCH in-charge',
    'Nursing officer in charge':'Nurse in-charge',
    'Deputy facility incharge':'Nurse in-charge',
    'Respective departmental leads ( MCH and maternity)':'Contact Person',
    'Nurse in-charge at MCH':'Nurse in-charge',
    'Incharge':'Nurse in-charge',
    'Nursing officer and the MCH officer in charge':'MCH in-charge',
    'MCH and Maternity in-charges respectively':'MCH in-charge',
    'Clinical officer/facility incharge':'Clinical Officer',
    'Facility matron':'MCH in-charge',
    '12%':'NA',
    'The MCH and Maternity in charges':'MCH in-charge',
    'incharge':'Nurse in-charge',
    'The MCH and Maternity in charges':'MCH in-charge',
    'NSM and MCH in charge after she resumes':'MCH in-charge',
    'The contact persons':'Contact Person',
    'Nursing officer in charge.':'Nurse in-charge',
    'Facility incharge,Mat incharge':'Nurse in-charge',
    'The facility nursing officer incharge':'Nurse in-charge',
    'The contact person. A nurse at the facility.':'Nurse',
    'MCH incharge\nMaternity incharge':'MCH in-charge',
    'Assistant in-charge':'Nurse in-charge',
    'Facility in-charge':'Nurse in-charge',
    'From now on wards the custodian will be the facility in charge, A nurse who is also our contact person.':'Nurse',
    'The person who will be the custodian from today onwards is the contact person who is the ANC nurse.':'Nurse',
    'MCH and maternity Nurse incharge':'Nurse in-charge',
    'MCH nurse/ nurse incharge':'Nurse in-charge',
    'Facility incharge (Nursing Officer)':'Nurse',
    'Facility in-charge.':'Nurse in-charge',
    'Nursing officer incharge':'Nurse in-charge',
    'ANC nurse.':'Nurse',
    'ANC and Maternity nurse':'Nurse',
    'nurse':'Nurse',
    'The facility NO incharge':'Nurse in-charge',
    'The Facility in charge will be the custodian from now hence forth.':'Contact Person',
    'The MCH in charge. She is also the current contact person':'MCH in-charge',
    'Nurse (the contact person who works at the ANC)':'Nurse',
    'The contact person.':'Contact Person',
    'The nursing officer incharge.':'Nurse in-charge',
    'MCH nurse and NO incharge':'MCH in-charge',
    'The MCH in-charge':'MCH in-charge',
    'NONE':'None',
    'Maternity incharge and MCH incharge.':'MCH in-charge',
    'Nursing service manager':'Nurse',
    'MCH and maternity incharges.':'MCH in-charge',
    'MCH and maternity incharges.':'MCH in-charge',
    'nsm':'Nurse',
    'MCH nurse and facility incharge':'Nurse',
    'ANC lead nurse':'Nurse',
    'ANC and Maternity in-charges':'MCH in-charge',
    'Contact persons':'Contact Person',
    'Facility Incharge ':'Nurse in-charge',
    'Facility Incharge':'Nurse in-charge',
    'the contact persons at ANC and Maternity':'Contact Person',
    'MCH INCHARGE':'MCH in-charge',
    'Facility Nurse people':'Nurse',
    'the contact person':'Contact Person',
    'MCH IN-CHARGE':'MCH in-charge',
    'Jane-the contact person':'Contact',
    'Nurse who is the contact person':'Contact Person',
    'ANC incharge':'Nurse in-charge',
    'nurses':'Nurse',
    'Nursing officer':'Nurse',
    'MCH and Maternity incharge.':'MCH in-charge',
    'Contact':'Contact Person',
    'The facility incharge':'Nurse in-charge',
    'Maternity and MCH in-charges respectively':'MCH in-charge',
    'AnC incharge':'Nurse in-charge',
    'Much and maternity in-charges':'MCH in-charge',
    'Respective departmental in-charges ( Maternity and MCH)':'MCH in-charge',
    'MCH incharge and maternity incharge':'MCH in-charge',
    'ANC and maternity nurse':'Nurse',
    'MCH nurse':'Nurse',
    'Facility incharge  and MCH nurse':'Nurse',
    'HRIO and facility incharge':'HRIO',
    'Mch nurse incharge':'Nurse in-charge',
    'Respective departmental in-charges':'Nurse in-charge',
    'Mch incharge and maternity Nurse':'MCH in-charge',
    'Nurses.':'Nurse',
    'Contact person a Nurse':'Nurse',
    'Nurse incharge':'Nurse in-charge',
    'MCH nurse incharge':'Nurse in-charge',
    'Facility n-charge':'Nurse in-charge',
    'MCH incharge, nutritionist and a maternity Nurse':'Nurse',
    'Mch nurse and maternity Nurse':'Nurse',
    'Data clerk.':'HRIO',
    'Facility Nurse':'Nurse',
    'MCH nurse incharge and maternity incharge':'Nurse in-charge',
    'MCH and Maternity in-charge.':'MCH in-charge',
    'MCH and Maternity incharge':'MCH in-charge',
    'NURSE':'Nurse',
    'MCH nurse and maternity Nurse':'Nurse',
    'ANC incharge and the Maternity Incharge.':'Nurse in-charge',
    'In-charge':'Nurse in-charge',
    'MCH and maternity Nurse':'Nurse',
    'MCH incharge and KCRHN':'MCH in-charge',
    'MCH nurse incharge and maternity Nurse incharge':'Nurse in-charge',
    'contact persons':'Contact Person',
    'the contact individuals':'Contact Person',
    'The MCH incharge.':'MCH in-charge',
    'Respective departmental nurses':'Nurse',
    'MCH Nurses and maternity Nurse':'Nurse',
    'Health records officer':'HRIO',
    '1.Health records officer \n2. Nurses':'HRIO',
    'Facility incharge Nurse and MCH nurse':'Nurse',
    'Facility MCH nurse':'Nurse',
    'Facility Nurse and facility incharge':'Nurse',
    'ANC nurses':'Nurse',
    'Facility CHV':'CHV',
    'The facility oncharge':'Contact Person',
    'MCH incharge and labour ward incharge and nutritionist':'MCH in-charge',
    'MCH nurse and maternity nurse':'Nurse',
    'CHB':'CHV',
    'Nurse in-charge maternity and MCH':'Nurse in-charge',
    'MCH incharge(Nurse) Deputy Maternity incharge(Nurse)':'Nurse',
    'Matron of the facility':'Matron',
    'Service provider who is on duty for the month':'Nurse in-charge',
    'MCH nurse.':'Nurse',
    'Facility Nurse.':'Nurse',
    'MCH incharge  and the contact person':'MCH in-charge',
    'MCH/Maternity incharge':'MCH in-charge',
    'Maternity and MCH incharges(nurses)':'Nurse',
    'Linda mama personel, Epsos staff and MCH Nurse':'Nurse',
    'Nursing Service Manager':'Nurse',
    'Facility incharge/Nurse':'Nurse in-charge',
    'Nursing Officer incharge':'Nurse in-charge',
    'DNSM':'Nurse',
    'The MCH incharge':'MCH in-charge',
    'Magdalene-contact person':'Contact Person',
    'Maternity and MCH incharge':'MCH in-charge',
    'The nursing officer incharge(nurse)':'Nurse in-charge',
    'The MCH and maternity in-charges':'MCH in-charge',
    'Hospital incharge':'Nurse in-charge',
    'Nurse MCH in-charge':'Nurse in-charge',
    'Facility Incharge.':'Nurse in-charge',
    'Maternity and MCH incharge.':'MCH in-charge',
    'Respective departmental contact persons':'Contact Person',
    'MCH in-charge and MNH Nurse':'MCH in-charge',
    'Mch Nurse incharge':'MCH in-charge',
    'Hospital administrator':'Contact Person',
    'Facility  incharge':'Contact Person',
    'MCH and ANC incharge':'MCH in-charge',
    'Mch/maternity in-charges':'MCH in-charge',
    'the facility incharge/contact person':'Contact Person',
    'MCH and maternity in charges':'MCH in-charge',
    'ANC- Contact person Colleta\nPNC-contact person Ruth Lumumba':'Contact Person',
    'Maternity CHV, MCH incharge':'MCH in-charge',
    'The MCH and the maternity incharge':'MCH in-charge',
    'ANC and PNC ward incharges':'Nurse in-charge',
    'The MCH and maternity incharge(nurses)':'Nurse in-charge',
    'The facility incharge(Nurse)':'Nurse in-charge',
    'Facility incharge-Clinical officer':'Clinical Officer',
    'ANC Nurse':'Nurse',
    'MCH and Maternity in-charges':'MCH in-charge',
    'MCH nurse and Maternity Nurse':'MCH in-charge',
    'Incharges':'Nurse in-charge',
    'Mentor mother':'Mentor Mother',
    "Nurse's":'Nurse',
    'Facility incharge.':'Nurse in-charge',
    'The facility incharge.':'Contact Person',
    'The facility incharge(CO)':'Clinical Officer',
    'contact person':'Contact Person',
    'Facility nurse incharge':'Nurse in-charge',
    'Maternity and MCH incharges':'MCH in-charge',
    'NURSES':'Nurse',
    'mentor mother':'Mentor Mother',
    'incharge MCH':'MCH in-charge',
    'Facility MCH incharge':'MCH in-charge',
    'Nursing officer incharge.': 'Nurse in-charge',
    'Facility Nurse incharge' : 'Nurse in-charge',
    'Nursing officer incharge_Nurse' :'Nurse in-charge',
    'The nursing officer incharge':'Nurse in-charge',
    'Facility nurse':'Nurse',
    'Hrio':'HRIO',
    'the contact persons at ANC and maternity':'Contact Person',
    'MCH incharge.' :'MCH in-charge',
    'Contact person':'Contact Person',
    'Facility incharge Nurse':'Nurse in-charge',
    'MCH nurse incharge and maternity Nurse':'Nurse in-charge',
    'MCH Nurse':'Nurse',
    'MCH in charge':'MCH in-charge',
    'the facility in charge':'Nurse in-charge',
    'MCH and Maternity incharges':'MCH in-charge',
    'MCH nurses':'Nurse',
    'the contact persons at the facility':'Contact Person',
    'Facility In-charge':'Nurse in-charge',
    'ANC in-charge':'Nurse',
    'MCH and maternity incharge':'MCH in-charge',
    'The facility medsup(Clinical Officer)':'Clinical Officer',
    'NUrses':'Nurse',
    'The Maternity and Mch incharges.':'MCH in-charge',
    'Nutrse':'Nurse',
    'Nursing Officer Incharge(Nurse)':'Nurse in-charge',
    'The facility Incharge(Clinical officer)':'Clinical Officer',
    'The facility matron(Nurse)':'Nurse',
    'Mch Nurses':'Nurse',
    'The facility nursing officer incharge.':'Nurse in-charge',
    'Matron':'Nurse',
    'MCH nurse and maternity incharge':'MCH in-charge',
    'ANC incharge and matenrity incharge':'Nurse in-charge',
    'Contact':'Contact Person',
    'Matron':'Nurse',


})

print(df_modified["Who is the custodian of the consent sheet at the facility?(Cadre)"].value_counts())

Nurse                 474
Nurse in-charge       323
                      196
MCH in-charge         154
Contact Person        124
Clinical Officer        9
HRIO                    8
Mentor Mother           8
N/A                     5
None                    4
NA                      3
Linda mama officer      3
CHV                     3
Contact                 1
Matron                  1
Name: Who is the custodian of the consent sheet at the facility?(Cadre), dtype: int64


In [ ]:
# What happens when this person is not around?


df_modified["What happens when this person is not around?"] = df_modified["What happens when this person is not around?"].str.strip().replace({
                    'Delegated to Assistant':'Some else is in charge',
                    'She handsover to her colleague nurse on duty':'Some else is in charge',
                    'Delegated to Assistant nurse':'Some else is in charge',
                    'the other nurses take over enrolment of mothers':'Some else is in charge',
                    'ANC nurse will take over':'Some else is in charge',
                    'Delegates the task to the nurse in duty':'Some else is in charge',
                    'Her colleague wi take over':'Some else is in charge',
                    'Nurse on duty takes charge':'Some else is in charge',
                    'The ANC nurse will be in custody of them':'Some else is in charge',
                    'incharges will take over':'Some else is in charge',
                    'Nurse at MCH':'Some else is in charge',
                    'Nurse on duty':'Some else is in charge',
                    'Deligate it to the nurse on duty':'Some else is in charge',
                    "Other nurses can still access the file and enroll mum's.":'Some else is in charge',
                    'Other nurses can still access the file and enroll':'Some else is in charge',
                    'She handsover to her colleague nurse for sensitization and enrollment':'Some else is in charge',
                    'Other Nurses can access the file  and enroll mums':'Some else is in charge',
                    'Delegate the task to the health records person':'Some else is in charge',
                    "Others can still enroll mum's":'Some else is in charge',
                    'ANC will take over':'Some else is in charge',
                    'The other nurses take over enrolment of mothers':'Some else is in charge',
                    'Other nurses can still access the file and enroll mums.':'Some else is in charge',
                    'Nurse':'Some else is in charge',
                    'Hand over to the nurse on duty':'Some else is in charge',
                    'The file will be handed over to the nurse on duty':'Some else is in charge',
                    'On weekends ANC register and consent sheets will locked in the CCC room.\n\nThe nurse on duty will retrieve both files for filling':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTS,consenting and assist mums to sign the forms.':'Some else is in charge',
                    'Other nurses can still continue with enrollments.':'Some else is in charge',
                    "Other nurses can still access the file and enroll mum's":'Some else is in charge',
                    'the other nurses take over':'Some else is in charge',
                    'Delegate the task to the nurse on duty':'Some else is in charge',
                    'Delegated to ANC nurse':'Some else is in charge',
                    'He handsover to a colleague nurse on shift':'Some else is in charge',
                    'Her colleague  Nurse':'Some else is in charge',
                    'The file will be handled by her colleague who stays in the facility compound':'Some else is in charge',
                    'All the nurses were ask to be responsible for safety and filling of the consent sheets':'Some else is in charge',
                    'others can acces the file and enroll mums.':'Some else is in charge',
                    'MCH incharge':'Some else is in charge',
                    'other nurses can still access the file and enroll mums.':'Some else is in charge',
                    'incharge will take over':'Some else is in charge',
                    'Anc nurse will take over':'Some else is in charge',
                    'The nurse handsover enrollment and custody to the nurse on duty':'Some else is in charge',
                    'Handsover to the colleague nurse on duty':'Some else is in charge',
                    'The MCH nurses are available throughout the week , for the maternity department nurses work in shifts therefore talk to mums about PROMPTS and assist to fill the consent sheets when on duty':'Some else is in charge',
                    'MCH Nurse on duty':'Some else is in charge',
                    'Nurse at MCH who is also our contact person':'Some else is in charge',
                    'the other nurses take over the enrolment of mothers':'Some else is in charge',
                    'Mentor mother will take over':'Some else is in charge',
                    'Deligate to the nurse on duty':'Some else is in charge',
                    'The nurse on duty':'Some else is in charge',
                    'The Nurse on duty will be responsible for safety of the consent sheet':'Some else is in charge',
                    'Delegate the task to the colleague':'Some else is in charge',
                    'Delegate the task to Mch and maternity nurses on duty':'Some else is in charge',
                    'His colleague will take over':'Some else is in charge',
                    "the rest can stil acess the file and enroll mums.":'Some else is in charge',
                    "Other nurses can access the file and enroll mums.":'Some else is in charge',
                    'MCH in charge will be the custodian':'Some else is in charge',
                    'Incharge will take over':'Some else is in charge',
                    'He handsover to a colleague nurse on duty':'Some else is in charge',
                    "Other Nurses can access the file and enroll mum's.":'Some else is in charge',
                    'The incharge will follow up to see that every nurse is task with filling the consent sheet while they are on duty at the MCH and Maternity department and hand over at the end of their shifts to the next staff.':'Some else is in charge',
                    'She hands over to the nurse on duty':'Some else is in charge',
                    'She handsover to her colleague nurse on shift to enroll mums and store the files':'Some else is in charge',
                    'Other persons can still acess the file and enroll mums':'Some else is in charge',
                    'MCH in charge':'Some else is in charge',
                    'Deligate it to nurse on duty':'Some else is in charge',
                    'She handsover to her colleague on duty':'Some else is in charge',
                    'Hands over to the nurse on duty':'Some else is in charge',
                    'Delegate the task a  colleague':'Some else is in charge',
                    'The ANC nurse will take over':'Some else is in charge',
                    'ANC incharge':'Some else is in charge',
                    'Delegated to the ANC nurse':'Some else is in charge',
                    'Delicate the task to the current nurse in charge':'Some else is in charge',
                    'Delegate the task to the current nurse on duty':'Some else is in charge',
                    'The other staff take over enrolment of mothers':'Some else is in charge',
                    'Deputy in-charge':'Some else is in charge',
                    'MCH in-charge':'Some else is in charge',
                    'The facility nurse in charge  will be taking over.':'Some else is in charge',
                    'The deputy in charges will be accountable':'Some else is in charge',
                    'She handsover to her colleague nurse':'Some else is in charge',
                    'Handsover to colleague nurse':'Some else is in charge',
                    'Deligate the task to the current nurse on duty':'Some else is in charge',
                    'Delegated to the contact person':'Some else is in charge',
                    'Deligate the duty to the nurse on duty':'Some else is in charge',
                    'the other nurses enroll mothers':'Some else is in charge',
                    'Delegated to Assistant in-charge':'Some else is in charge',
                    'the colleagues take over the enrolment of mothers':'Some else is in charge',
                    'the other nurses are aware of the program and so they can send data':'Some else is in charge',
                    'MCH and maternity incharge will be responsible.':'Some else is in charge',
                    'Nurse She hands over the enrollments to the nurse on duty':'Some else is in charge',
                    'The Staff on duty talks to Mums about PROMPTS':'Some else is in charge',
                    "Other nurses can access the file and enroll mum's.":'Some else is in charge',
                    'ANC nurse':'Some else is in charge',
                    'Nurse working at MCH':'Some else is in charge',
                    'the other nurses take over the enrolments':'Some else is in charge',
                    'Other nurses can still access the forms and enroll.':'Some else is in charge',
                    'The deputy will take charge':'Some else is in charge',
                    'The clinical officer who has been our contact person will be the custodian':'Some else is in charge',
                    'Delegate the work to the nurse on duty at the ANC clinc':'Some else is in charge',
                    'The staff on duty takes the responsibility of talking to mothers about PROMPTS':'Some else is in charge',
                    'The deputies will be incharge':'Some else is in charge',
                    'Other nurses can still access the file.':'Some else is in charge',
                    "Other Nurses can still access the file and enroll mum's.":'Some else is in charge',
                    "Others can still access the file and enroll mum's.":'Some else is in charge',
                    'the other nurses take over thetask':'Some else is in charge',
                    'Delegated to MCH  in-charges':'Some else is in charge',
                    'Delegate the custody to the nurse at the station.':'Some else is in charge',
                    'Delegate the duty to the  nurse on duty':'Some else is in charge',
                    "The staff on duty  takes the responsibility of enrolling mother's via consent forms":'Some else is in charge',
                    'the other nurses help to enroll mothers':'Some else is in charge',
                    'Her colleague  will take over':'Some else is in charge',
                    'Handsover to the nurse on duty':'Some else is in charge',
                    'The deputy will be the custodian':'Some else is in charge',
                    'the other nurses take over the duty':'Some else is in charge',
                    'Delegated to respective department in-charges':'Some else is in charge',
                    'Facility in-charge will be the custodian':'Some else is in charge',
                    'the colleagues take over the tasks':'Some else is in charge',
                    'She hands over to the nursing officer incharge.':'Some else is in charge',
                    'Hands over to colleague nurse on shift':'Some else is in charge',
                    'Delegated to the assistant':'Some else is in charge',
                    'others can still access the file and enroll mums.':'Some else is in charge',
                    'He hands over to the nurse on duty':'Some else is in charge',
                    'The staff on duty will be responsible for the consent sheets.':'Some else is in charge',
                    'Delegated to respective assistants':'Some else is in charge',
                    'The contact persons to take charge':'Some else is in charge',
                    'Hands over to the nurse on shift':'Some else is in charge',
                    'other nurse can still continue enrolling mums.':'Some else is in charge',
                    'the colleagues take over enrolment of mothers':'Some else is in charge',
                    'The other ANC nurse will take over':'Some else is in charge',
                    'Other nurses can still access and enroll mums.':'Some else is in charge',
                    'Nurses':'Some else is in charge',
                    'Anc and PNC nurses will take over':'Some else is in charge',
                    'Delegated to each department in-charges ( MCH and Maternity':'Some else is in charge',
                    'Delegates the task to the colleague':'Some else is in charge',
                    'Other nurses can still access the file and Enroll.':'Some else is in charge',
                    'She hands over to the MCH and Maternity department.':'Some else is in charge',
                    'Nurse (contact person)':'Some else is in charge',
                    'the other nurses take up enrolment of mothers':'Some else is in charge',
                    'Deligate the task to nurse on duty':'Some else is in charge',
                    'The nurse on duty takes over':'Some else is in charge',
                    'the other nurses/records officers take over the task':'Some else is in charge',
                    'Other Nurses can access the file and enroll.':'Some else is in charge',
                    'Other Nurses can access  the file and continue with enrollment.':'Some else is in charge',
                    'Those working in the MCH and Maternity to take charge':'Some else is in charge',
                    'Delegate the task to the current nurse on duty at that time':'Some else is in charge',
                    'Other nurses can still access the file and enroll mums':'Some else is in charge',
                    'Facility in-charge':'Some else is in charge',
                    'He handsover to the nurse on duty':'Some else is in charge',
                    'Other nurses can access the forms and enroll':'Some else is in charge',
                    'The staff appointed as a representative will be in charge.':'Some else is in charge',
                    'the other staff talk to mothers about the program':'Some else is in charge',
                    'The supporting staff':'Some else is in charge',
                    'the colleagues take over the task':'Some else is in charge',
                    'Nurse at CWC':'Some else is in charge',
                    'The nursing officer will be responsible.':'Some else is in charge',
                    'Nurse stationed at MCH':'Some else is in charge',
                    'other nurses can still access the file and enroll mums':'Some else is in charge',
                    'She hands over to a colleague on duty':'Some else is in charge',
                    'Other Nurses can access the file and enroll mums':'Some else is in charge',
                    'MCH nurse':'Some else is in charge',
                    'the other staff are aware of the program and so they take over when one is absent':'Some else is in charge',
                    'Other nurses can still access the files and register mums':'Some else is in charge',
                    'The nurse in station/duty will take over':'Some else is in charge',
                    'Deligate it to the Hrio':'Some else is in charge',
                    'AnC nurse will take over':'Some else is in charge',
                    'Mch nurses to be incharge.':'Some else is in charge',
                    'Delegate the task to the nurse in station':'Some else is in charge',
                    'Facility incharge Nurse':'Some else is in charge',
                    'The person who is on duty will take care of the files and consent sheets.':'Some else is in charge',
                    'Other nurses can still access the file and win.':'Some else is in charge',
                    'Hospital Administration':'Some else is in charge',
                    'Other nurse can still access the file and enroll mums.':'Some else is in charge',
                    'Mch nurses will be incharge.':'Some else is in charge',
                    'He handsover to a colleague nurse':'Some else is in charge',
                    'NSM will take over':'Some else is in charge',
                    'Nurse at ANC':'Some else is in charge',
                    'Her colleague  will help':'Some else is in charge',
                    'The assistant facility incharge Nurse':'Some else is in charge',
                    'Delegated to assisting nurse':'Some else is in charge',
                    'Hands over to her colleague nurse on duty':'Some else is in charge',
                    'Other colleague nurses are incharge of sensitization and enrollment':'Some else is in charge',
                    'NSM':'Some else is in charge',
                    'The nurse on duty will be responsible for safety/ and filling of the consent sheets':'Some else is in charge',
                    "Other Nurses can still access the file and enroll mum's":'Some else is in charge',
                    'the colleagues take over':'Some else is in charge',
                    'The mat nurse will take charge':'Some else is in charge',
                    'The deputies or NSM will be the custodian':'Some else is in charge',
                    'Delegated to the respective contact persons':'Some else is in charge',
                    'Maternity staffs will be responsible':'Some else is in charge',
                    'She will be handing over to her colleague on duty':'Some else is in charge',
                    'Delegated to assistant':'Some else is in charge',
                    'the other colleagues take over enrolment of mothers':'Some else is in charge',
                    "Other nurses can access the file and enroll mum's":'Some else is in charge',
                    'the other nurses take over the task':'Some else is in charge',
                    'The othe colleague  will take over':'Some else is in charge',
                    'The are 2 in MCH her colleague  will take over':'Some else is in charge',
                    'A representative will be appointed by the incharge':'Some else is in charge',
                    'MCH and Maternity in-charge':'Some else is in charge',
                    'She hands over to her colleagues on duty':'Some else is in charge',
                    'Other nurses can access the file and enroll.':'Some else is in charge',
                    'He handsover to his colleague nurse on duty':'Some else is in charge',
                    'their colleagues will take over':'Some else is in charge',
                    'Others can still access the file and enroll mums.':'Some else is in charge',
                    'all the other nurses are aware of the program and so can run the enrolment of mothers':'Some else is in charge',
                    'Other nurses can access the file and continue with enrollment.':'Some else is in charge',
                    'MCH incharge will be available to take over.':'Some else is in charge',
                    'Appoint a colleague to be the custodian.':'Some else is in charge',
                    'Hand over is done to the MCH side which coordinates the enrollments for the entire facility.':'Some else is in charge',
                    'Other nurses are able to access the file and continue with enrollment.':'Some else is in charge',
                    'None':'Not sure',
                    'Delegate the work to the staff on duty':'Some else is in charge',
                    'He handsover to his colleague on duty':'Some else is in charge',
                    'the other nurses are aware of the program and can support enrolment of mothers':'Some else is in charge',
                    'Delegate the work to the person in duty.':'Some else is in charge',
                    'Facility In-charge':'Some else is in charge',
                    'She delegates to her colleague on duty':'Some else is in charge',
                    'He hand over.':'Some else is in charge',
                    'She hands it over to the facility incharge':'Some else is in charge',
                    'MCH and mat incharge will be the custodians.':'Some else is in charge',
                    'nurses can access the file and help mums enroll.':'Some else is in charge',
                    'Her colleague':'Some else is in charge',
                    'the others nurses take over':'Some else is in charge',
                    'They will be appointing the staff on duty to take care of them.':'Some else is in charge',
                    'Assistant nurse':'Some else is in charge',
                    'Delegate the task to the current nurse incharge':'Some else is in charge',
                    'Nurses in those departments will be responsible.':'Some else is in charge',
                    'Leave them to a CHV to continue enrolling':'Some else is in charge',
                    'SCPHN who has an office at the facility':'Some else is in charge',
                    'Hand over to the one who is on duty':'Some else is in charge',
                    'The consent sheets are handed to whoever is on duty':'Some else is in charge',
                    'Administrator':'Some else is in charge',
                    'The facility incharge will facilitate/ follow up to ensure mothers are enrolled via consent sheets and that the file is stored in a secure room.':'Some else is in charge',
                    'The staff on duty helps enroll mothers via consent forms':'Some else is in charge',
                    'contact person will take over':'Some else is in charge',
                    'NSM will be the custodian':'Some else is in charge',
                    'others can access the file and enroll mums':'Some else is in charge',
                    'the others nurses take over the task':'Some else is in charge',
                    'The nurses on duty':'Some else is in charge',
                    'The Nurse on duty will ensure mothers fill the consent sheets':'Some else is in charge',
                    'Hands over to the colleague nurse on duty  for custody, sensitization and enrollment':'Some else is in charge',
                    'The staff on duty helps to fill mums details on the consent forms':'Some else is in charge',
                    'Incharge will  take over':'Some else is in charge',
                    "other nurses can access the file and enroll mum's":'Some else is in charge',
                    'The facility RH and the facility incharge takes the lead':'Some else is in charge',
                    'They will appoint a representative and inform me':'Some else is in charge',
                    'Deputy MCH in-charge will be the custodian':'Some else is in charge',
                    'NSM when MCH in charge is not available \nMCH in charge when NSM is not available':'Some else is in charge',
                    'The nurse on duty will  be tasked to assist mums in filling the consent sheets':'Some else is in charge',
                    'The incharge takes the responsibility of helping mums enroll via consent forms':'Some else is in charge',
                    'The nurse in duty is delegated the task.':'Some else is in charge',
                    'Nurses in the mch takes charge':'Some else is in charge',
                    'Nurses at MCH and Maternity will be the custodian':'Some else is in charge',
                    'Deligate the task to the nurse on duty at that time':'Some else is in charge',
                    'The mch incharge takes over':'Some else is in charge',
                    'The nurse on duty will  be responsible  for safety and  assist mums in filling the consent sheets':'Some else is in charge',
                    'He will be dedicating the work to his colleague on duty.':'Some else is in charge',
                    'She hands over to colleagues as was handed over to her by the facility incharge':'Some else is in charge',
                    "The other mentor mother can still access the file and enroll mum's.":'Some else is in charge',
                    'Their deputies will be incharge':'Some else is in charge',
                    'the other nurses at the facility':'Some else is in charge',
                    'The incharge will be responsible to filling and safety of the consent sheets.':'Some else is in charge',
                    'the other colleagues':'Some else is in charge',
                    'She will be assigning the custody to the nurse on duty at MCH.':'Some else is in charge',
                    'Deligate it to whoever was on duty':'Some else is in charge',
                    '10%':'Not sure',
                    'A mentor mother will take over':'Some else is in charge',
                    'They hand over to the nursing officer incharge':'Some else is in charge',
                    'The n.o hands over':'Some else is in charge',
                    'Deligate the task to the nurse on duty':'Some else is in charge',
                    'The files will be filled by the nurse on duty as they fill other documents in the facility':'Some else is in charge',
                    'the other staff ake over enrolment of mothers':'Some else is in charge',
                    'the other nurses are aware of the program and can run it':'Some else is in charge',
                    'Not Sure':'Not sure',
                    'Other Nurses can still access the file and enroll mums.':'Some else is in charge',
                    'Hands over to mch and maternity incharges':'Some else is in charge',
                    'We have two contacts persons who alternates at work. If one is not around the other takes over':'Some else is in charge',
                    'MCH nurse on duty':'Some else is in charge',
                    'The duty is delegated to whoever was incharge':'Some else is in charge',
                    'Delegates to a colleague':'Some else is in charge',
                    'Handsover to a colleague nurse on duty for enrollment and custody':'Some else is in charge',
                    'Nurses who are doing the enrollment will take over':'Some else is in charge',
                    'Facility Incharge':'Some else is in charge',
                    'The file is stored in a safe cabinet accessible to all mothers':'Some else is in charge',
                    'Their deputies will be the custodian':'Some else is in charge',
                    'NsM':'Some else is in charge',
                    'Staff serving at MCH will be the custodian.':'Some else is in charge',
                    'The MCH in-charge who will be joining shall be the custodian':'Some else is in charge',
                    'mentor mother will take over':'Some else is in charge',
                    'the other nurses take over enrolmemt of  mothers':'Some else is in charge',
                    'Nurses in the maternity and MCH takes over.':'Some else is in charge',
                    'She handsover to the nurse on duty':'Some else is in charge',
                    'Their deputies will be the custodians':'Some else is in charge',
                    'The health records staff will be taking care of the consent sheets.':'Some else is in charge',
                    'She will be appointing her colleague -ANC nurse to take care of them and any matter related to prompts':'Some else is in charge',
                    'She will be assigning the nurse on duty to take over the custody of the consent sheets.':'Some else is in charge',
                    'Others should be able to access the file and enroll mums.':'Some else is in charge',
                    'Nurse at ANC will be the custodian':'Some else is in charge',
                    'Other Nurses can still access the file and enroll mums':'Some else is in charge',
                    'The facility incharge will be made aware':'Some else is in charge',
                    'The nurse on duty will talk to mothers about PROMPTs and help the mothers sign the consent forms':'Some else is in charge',
                    'The mentor mother will talking about PROMPTs Mothers who are waiting to be served, and the nurse on duty will help the mothers fill and sign the consent sheets.':'Some else is in charge',
                    'He will be appointing a custodian to take care of the consent sheets.':'Some else is in charge',
                    'Hands over to her colleague nurse/on duty or facility incharge':'Some else is in charge',
                    'They hand over to colleagues in the departments.':'Some else is in charge',
                    'Nurses are able to access the file and enroll mums.':'Some else is in charge',
                    'mentor mother  will take over':'Some else is in charge',
                    'Other nurses  can still continue with enrollment.':'Some else is in charge',
                    'The other colleagues in the MCH on duty':'Some else is in charge',
                    'The appointed person in the mch will be incharge':'Some else is in charge',
                    'Nurse who works at the consultation room. They rotate within the departments':'Some else is in charge',
                    'MCH in charge will take over':'Some else is in charge',
                    'Delegate the task to the current nurse in charge.':'Some else is in charge',
                    'delegated to CHV':'Some else is in charge',
                    'The Nurse on duty will talk to mums and help them fill consent forms.':'Some else is in charge',
                    'other nurses can still continue with the enrollments':'Some else is in charge',
                    'Delegated to Linda Mama officer':'Some else is in charge',
                    'Orientate all post natal the staff to ensure that when they are on duty they are filling the sheets.\n\nThe nurse at the new ANC client room mans the room every day unless she is on leave therefore she will be filling the consent sheets everyday':'Some else is in charge',
                    'The nurse on duty will be the responsible for the consent sheets file storage':'Some else is in charge',
                    'The MCH nurse takes charge':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTs and assist mothers to fill the consent forms':'Some else is in charge',
                    'delegated to assistant':'Some else is in charge',
                    'The Staff on  duty will  talk to mothers about PROMPTS and consenting':'Some else is in charge',
                    "other nurses can still access the file and enroll mum's":'Some else is in charge',
                    'Delegated to the respective CHVs':'Some else is in charge',
                    'Mch incharge will take charge.':'Some else is in charge',
                    'The nurse  on duty will be responsible for the consent sheets':'Some else is in charge',
                    'Another ANC nurse will take over':'Some else is in charge',
                    'The staff on duty enroll mums via consent forms':'Some else is in charge',
                    'The staff on duty will talk to mums about Prompts and help to enroll them.':'Some else is in charge',
                    'COVID casual staff.':'Some else is in charge',
                    'MCh and Maternity incharges':'Some else is in charge',
                    'She will assign someone to be the custodian':'Some else is in charge',
                    'Delegated to assist in-charge':'Some else is in charge',
                    'MCH in-charge will be the custodian':'Some else is in charge',
                    'The incharge  will ensure the next staff on duty takes custody of the consent sheets.':'Some else is in charge',
                    'Nurse MCH incharge':'Some else is in charge',
                    'Nurse in the MCH becomes the custodian.':'Some else is in charge',
                    'Deligate the duty to either of nurse who is on duty.':'Some else is in charge',
                    'The nurse incharge takes over':'Some else is in charge',
                    'Delegated to nurse':'Some else is in charge',
                    'The other staff they work with at MCH':'Some else is in charge',
                    'Appointed MCH and maternity staffs will be responsible':'Some else is in charge',
                    'She hands over to the colleague nurse on duty':'Some else is in charge',
                    'other staff at the MCH and maternity support with enrollment':'Some else is in charge',
                    'Deligate the duty to nurses on duty':'Some else is in charge',
                    'Chv will take over':'Some else is in charge',
                    'The other ANC will take over':'Some else is in charge',
                    'The file will be stored next to where ANC registers are stored and can be retrieved and returned at the end of the shift by the staff on duty.\n\nAt the maternity department, the file will be at the nurse station and handed over to  the next nurse on sift during handing over.':'Some else is in charge',
                    'The incharge will ensure  that all the staff  take  responsibility while on duty to fill the consent sheets and ensure its safety.':'Some else is in charge',
                    'Deligate the duty to the nurse on duty.':'Some else is in charge',
                    'Delegated to the other nurses at MCH and maternity respectively':'Some else is in charge',
                    'She delegates':'Some else is in charge',
                    'Deligate it to colleagues in the MCH and maternity':'Some else is in charge',
                    'Deligate it to the facility HRIO':'Some else is in charge',
                    'The staff on duty will help mothers consent':'Some else is in charge',
                    'Other nurses can still continue with the enrollment.':'Some else is in charge',
                    'The contact person will be dedicated the task to the colleague on duty.':'Some else is in charge',
                    'the other facility staff are also aware of the program and are willing to support':'Some else is in charge',
                    'the other colleagues take over':'Some else is in charge',
                    'MCH and maternity incharges become the custodians.':'Some else is in charge',
                    'Handover done to the nurses in the mch who also doubles for maternity side.':'Some else is in charge',
                    'Deligating to the nurse on duty':'Some else is in charge',
                    'the other nurses enroll the mothers':'Some else is in charge',
                    'the other nurses take over the task of enrolling mothers':'Some else is in charge',
                    'enrollment still goes on as other can still access the forms.':'Some else is in charge',
                    'the other 3 colleagues can take over':'Some else is in charge',
                    'their colleagues in respective departments take over':'Some else is in charge',
                    'The facility has embraced an inclusive approach therefore anyone is eligible for being the custodian of the consent.':'Some else is in charge',
                    "Nurse who will be assigned to work in the department when the in-charge isn't around":'Some else is in charge',
                    'Handover is done to the nurse in the mch and maternity':'Some else is in charge',
                    'Deligate it to the one on duty':'Some else is in charge',
                    'The current nurse on duty takes over':'Some else is in charge',
                    'Other nurses can still access the consent forms and continue with enrollments.':'Some else is in charge',
                    'MCH and Maternity in-charges will be the custodians':'Some else is in charge',
                    'Mentor  father will take over':'Some else is in charge',
                    'Delegate the task to the nurse incharge.':'Some else is in charge',
                    'The facility nurse incharge':'Some else is in charge',
                    'She hands over to MCH and maternity incharges':'Some else is in charge',
                    'The nurse on duty will take up the task':'Some else is in charge',
                    'NSM will follow up but the incharges should ensure  the program is running in either departments':'Some else is in charge',
                    'Other Nurses can still enroll mums':'Some else is in charge',
                    'Handsover to a colleague nurse for sensitization and storage':'Some else is in charge',
                    'Handover to the colleague nurse on duty':'Some else is in charge',
                    'Mentor mother':'Some else is in charge',
                    'He handsover to his colleague':'Some else is in charge',
                    'The staff on duty will talk to the mothers and help them consent.':'Some else is in charge',
                    'Delegate the duty to the nurse on duty':'Some else is in charge',
                    'Delegated to the maternity in-charge':'Some else is in charge',
                    'The other MCH staff will be the custodian':'Some else is in charge',
                    'The maternity in-charge will be the custodian':'Some else is in charge',
                    'The file is still accessible and enrollments go on.':'Some else is in charge',
                    'other nurses are in the position to enroll and consent mums.':'Some else is in charge',
                    'the others ANC nurse will take over':'Some else is in charge',
                    'staff on duty takes the responsibility to talk to mums about PROMPTs':'Some else is in charge',
                    'Delegated to the Assistant':'Some else is in charge',
                    'Facility incharge will ensure the form is filled by the nurse on duty':'Some else is in charge',
                    'The contact  person will coordinate with the facility  incharge to see that forms are well filled and sent to Jacaranda for processing.':'Some else is in charge',
                    'It is deligated to whoever is on duty':'Some else is in charge',
                    'the other facility staff take over the job':'Some else is in charge',
                    'Nurse at MCH(ANC) will be the custodian':'Some else is in charge',
                    'anc nurse will take over':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTS':'Some else is in charge',
                    'other nurses can access the file and enroll the mums.':'Some else is in charge',
                    'Staff on duty will ensure that mums are talked to about PROMPTS and assist the mother fill the consent sheet':'Some else is in charge',
                    'The other colleagues at the unit take over':'Some else is in charge',
                    'MCH Nurse':'Some else is in charge',
                    'others should be able to continue with enrollment.':'Some else is in charge',
                    'Delegate the task to the nurse on duty.':'Some else is in charge',
                    'The staff on duty will assist mums  to fill and sign the consent forms':'Some else is in charge',
                    'Deligate the duty to a nurse on duty':'Some else is in charge',
                    "Other nurse can access the file and enroll mum's.":'Some else is in charge',
                    'Persons working in the two departments will be nominated.':'Some else is in charge',
                    'The mch incharge will be the custodian':'Some else is in charge',
                    'others can access and enroll mums.':'Some else is in charge',
                    'Others can still access the file and enroll mums':'Some else is in charge',
                    'Other nurses can still acess the file and enroll mums':'Some else is in charge',
                    'Delegate the nurse to the nurse on station.':'Some else is in charge',
                    'He hands over to the MCH and Maternity incharge':'Some else is in charge',
                    'Delegated to a nurse in respective department':'Some else is in charge',
                    'A person in the MCH and Maternity will nominated to take charge.':'Some else is in charge',
                    'Nurse at MCH in charge of GANC group':'Some else is in charge',
                    'The nurse who they work with at MCH':'Some else is in charge',
                    'The nursing officer of the facility takes charge.':'Some else is in charge',
                    'Staff on shift will talk to mothers about PROMPTS and keep the forms safe':'Some else is in charge',
                    'the nurses work as a team and in absence of the contact person the colleagues take over':'Some else is in charge',
                    'Nurses in the two departments take charge.':'Some else is in charge',
                    'Nurse on duty will be tasked to talk to mums':'Some else is in charge',
                    'MCH nurse will be the custodian':'Some else is in charge',
                    'She handsover to her colleague nurse on shift':'Some else is in charge',
                    "Others can access the file and enroll mum's.":'Some else is in charge',
                    'She handsover to a colleague on duty':'Some else is in charge',
                    'The MCH nurse':'Some else is in charge',
                    'The nurse they serve with will be the custodian':'Some else is in charge',
                    'The facility has lockable cabinets that any other healthcare provider in the maternity and MCH can access therefore become custodian of the sheets.':'Some else is in charge',
                    'Handsover the custody and responsibility of enrolling the mums to the colleague on duty':'Some else is in charge',
                    'Handsover to the colleague on duty for storage and the role of enrollment':'Some else is in charge',
                    'Clerk will take over':'Some else is in charge',
                    "Clinical officer who's the contact person of the facility will be incharge":'Some else is in charge',
                    'They deligate to the nurse on duty':'Some else is in charge',
                    'the other ANC  nurse will take over':'Some else is in charge',
                    'The facility incharge':'Some else is in charge',
                    'Nurses in the MCH and maternity will take charge.':'Some else is in charge',
                    'Left to the in-charge':'Some else is in charge',
                    'ANC Nurse':'Some else is in charge',
                    'Their deputies':'Some else is in charge',
                    'Mch incharge of the facility will be the custodian':'Some else is in charge',
                    'Nurse She hands over to the nurse on duty':'Some else is in charge',
                    'Contact person is the custodian':'Some else is in charge',
                    'Nurse who is our contact person':'Some else is in charge',
                    'The files are kept in mch and maternity cabinets therefore nurses in the two departments become custodians if the incharge is not around.':'Some else is in charge',
                    'The mentor mother who will be helping mothers consent will be incharge':'Some else is in charge',
                    'The staff they work with at MCH':'Some else is in charge',
                    'Other Nurses can still enroll.':'Some else is in charge',
                    'The MCH nurses will be incharge':'Some else is in charge',
                    'Deligate the duty to the nurse incharge':'Some else is in charge',
                    'She handsover to the facility Incharge':'Some else is in charge',
                    'She handsover to her colleague':'Some else is in charge',
                    'The facility incharge become the custodian.':'Some else is in charge',
                    'The nurse working at MCH will take over':'Some else is in charge',
                    'She will appoint one of MCH in charge to be the custodian':'Some else is in charge',
                    'The MCH incharge  will be incharge.':'Some else is in charge',
                    'Facility Incharge.':'Some else is in charge',
                    'They deligate to nurse on duty':'Some else is in charge',
                    'deligate it to Nurse in duty':'Some else is in charge',
                    'nurse (who is the contact person)':'Some else is in charge',
                    'delegate the task to the nurse on duty.':'Some else is in charge',
                    'Hands over to the facility Incharge who is also responsible for sensitizing and enrolling the mums':'Some else is in charge',
                    'Nurse at the MCH will take over':'Some else is in charge',
                    'The inchagre will take over':'Some else is in charge',
                    'Mch incharge will be liable':'Some else is in charge',
                    "Other nurse can still access the file and enroll mum's":'Some else is in charge',
                    'The files are put in an accessible rooms(Anc and maternity)':'Some else is in charge',
                    'The other colleagues take over enrolment of mothers':'Some else is in charge',
                    'Other nurses are able to access the file and enroll mums.':'Some else is in charge',
                    'Other nurses can still access the forms':'Some else is in charge',
                    'Everyone working in the MCH is to be informed and take responsibility.':'Some else is in charge',
                    'Deligate it to the nurse incharge':'Some else is in charge',
                    'All nurses on duty will  be responsible to talk to the mothers in maternity and mch department.':'Some else is in charge',
                    'Nurse at MCH (contact person) is the custodian':'Some else is in charge',
                    'Any of the staff in attendence. \nNurse or hrio':'Some else is in charge',
                    'Nurse on duty or the HRIO':'Some else is in charge',
                    'Her colleague takes over the sensitization and enrollment of mums':'Some else is in charge',
                    'They will delegate to their deputies':'Some else is in charge',
                    'Deligate it to nurse on duty.':'Some else is in charge',
                    'Deligates the task to nurse on duty.':'Some else is in charge',
                    'The staff on duty will  be responsible for helping mothers consent.':'Some else is in charge',
                    'ANc  nurse will take over':'Some else is in charge',
                    'Deligate it to MCH Nurse':'Some else is in charge',
                    'The staff on duty talks to mums and enrolls them via consent forms':'Some else is in charge',
                    'delegated to Assistant':'Some else is in charge',
                    'All the staff will be responsible for the custody of the sheets, ensure they are fill and kept safe when they are on duty.':'Some else is in charge',
                    'The incharge will request person on duty to be the custodian of the sheets and fill mothers details':'Some else is in charge',
                    'Another colleague, takes over sensitization and enrollment':'Some else is in charge',
                    'the other colleagues are aware of the program and can enroll mothers':'Some else is in charge',
                    'Nurse no duty fills the consent foms':'Some else is in charge',
                    'The nurse incharge yes':'Some else is in charge',
                    'Deligate it to the casual nurse .':'Some else is in charge',
                    'She hands over to maternity and mch incharges.':'Some else is in charge',
                    'The incharge will ensure the staff on duty is responsible for the consent sheets':'Some else is in charge',
                    'The staff on duty is tasked with helping the mothers fill and sign the consent sheets':'Some else is in charge',
                    'The nurse is expected to hand over the consent sheets to the next nurse on duty.for data collection and safe keeping.':'Some else is in charge',
                    'The mch incharge will be responsible':'Some else is in charge',
                    'She hands over to the mch and maternity incharges.':'Some else is in charge',
                    'Nurse will be the custodian':'Some else is in charge',
                    'They handover to their colleagues on duty':'Some else is in charge',
                    'her workmate will take over':'Some else is in charge',
                    'Other nurses can still enroll mums.':'Some else is in charge',
                    'Another person will be nominated':'Some else is in charge',
                    'Two of the colleagues(at the Anc and Maternity)will be taking over.':'Some else is in charge',
                    'The maternity incharge will be responsible.':'Some else is in charge',
                    'She hands over to the staffs in the MCH and maternity':'Some else is in charge',
                    'He will Deligate it to the the nurse on duty':'Some else is in charge',
                    'The facility incharge Nurse will deligate':'Some else is in charge',
                    'ANc will will take over':'Some else is in charge',
                    'The mch incharge will take charge.':'Some else is in charge',
                    'Other nurses can still access the file and continue with the enrollment.':'Some else is in charge',
                    'Other health care workers are able to continue with enrollment.':'Some else is in charge',
                    'the other staff take over':'Some else is in charge',
                    "Others can still access the file and enroll mum's":'Some else is in charge',
                    'She handsover to the colleague on duty':'Some else is in charge',
                    'Let the nurse on station take care of them.':'Some else is in charge',
                    'The nurse on duty is responsible for the consent forms and talking to the mother about PROMPTS and consenting.':'Some else is in charge',
                    'Delegated to contact person':'Some else is in charge',
                    'Delegated to Assistant health records officer':'Some else is in charge',
                    'Delagated to the other ANC nurse.':'Some else is in charge',
                    'Providers in the mch and maternity will be nominated.':'Some else is in charge',
                    'Deligate to fellow nurse':'Some else is in charge',
                    'The staff agreed to help mums fill the forms when on duty':'Some else is in charge',
                    'Nurses can still access the file and enroll mums.':'Some else is in charge',
                    'Deligate it to the MCH nurse':'Some else is in charge',
                    'The staff agreed to help mums fill the forms when on duty':'Some else is in charge',
                    'Nurses can still access the file and enroll mums.':'Some else is in charge',
                    'Deligate it to the MCH nurse':'Some else is in charge',
                    'All nurses  agreed to be talking to mums and fill the consent  sheets while  they are on duty':'Some else is in charge',
                    'Delegate the duty to the current nurse in charge':'Some else is in charge',
                    'The other nurses continue to enroll mothers':'Some else is in charge',
                    'the contact persons also have access to the sheets':'Some else is in charge',
                    'The deputy at MCH and maternity will be the custodians':'Some else is in charge',
                    'Hands over to a colleague to continue sensitization and enrollments':'Some else is in charge',
                    'Hand over to colleague nurse to sensitize and enroll mums':'Some else is in charge',
                    'her colleague will take over':'Some else is in charge',
                    'Deligate it to cwc nutritionist':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTs':'Some else is in charge',
                    'The files are handed over to the nurses on shift from the respective departments.':'Some else is in charge',
                    'Others nurses can  still enroll mums':'Some else is in charge',
                    'ANC Nurse will take over':'Some else is in charge',
                    'The other nurses are aware of the programme so they take over the activity':'Some else is in charge',
                    'The file is given to the nursing officer incharge.':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTs and assist mothers to enroll via consent forms.':'Some else is in charge',
                    'The staff on duty will be the custodian of the consent sheets':'Some else is in charge',
                    'The staff  on duty becomes the custodian of the consent sheets':'Some else is in charge',
                    'The Nurse on duty will be responsible to helping mothers consent':'Some else is in charge',
                    'He hands over the file to the  nurse covering':'Some else is in charge',
                    'The MCH incharge will be responsible.':'Some else is in charge',
                    'Delegated to Assistant admin and contact person':'Some else is in charge',
                    'The MCH nurse who is the contact person takes care of the files':'Some else is in charge',
                    'The nursing officer incharge takes charge':'Some else is in charge',
                    'Deligating the duty to nurse on duty or to an Epsos':'Some else is in charge',
                    'The nursing officer hands over to the mch and maternity incharge.':'Some else is in charge',
                    'Nurse (our contact person)':'Some else is in charge',
                    'She/he deligates to whoever is on duty':'Some else is in charge',
                    'Delegate to Linda mama officer who is also the contact person':'Some else is in charge',
                    'Delegate to Assistant nurse':'Some else is in charge',
                    'Delegated to the CHV':'Some else is in charge',
                    'She hands over to the staffs in the MCH or maternity.':'Some else is in charge',
                    'Deligates to the nurse on duty . There are only two nurses in the facility':'Some else is in charge',
                    'Facility Incharge will task a staff to talk to the mothers about PROMPTs':'Some else is in charge',
                    'Nurse on duty will help fill the consent forms':'Some else is in charge',
                    'Other nurses continue with enrollment':'Some else is in charge',
                    'Enrollment goes on as all nurses are in apposition to access the  PROMPTS file.':'Some else is in charge',
                    'The MCH incharge will take charge of it.':'Some else is in charge',
                    'The other nurses on duty take over':'Some else is in charge',
                    'The nurse on duty will talk to mums about the program and help them enroll.':'Some else is in charge',
                    'Mch nurse or cwc nurse':'Some else is in charge',
                    'The staff on duty talks to mums and helps them consent.':'Some else is in charge',
                    'The staff on duty talks to mums about PROMPTS':'Some else is in charge',
                    'The Nrse who is doing the registration  will pick it up':'Some else is in charge',
                    'Deligate it to the nurse that will be on duty':'Some else is in charge',
                    'Had a meeting with the other staff and they agreed to assist in case in absentia':'Some else is in charge',
                    'MCH and maternity incharge are to take charge.':'Some else is in charge',
                    'Her colleague is responsible for sensitization, enrollment and storage of consent sheets':'Some else is in charge',
                    'The nurse on duty will be the custodian of the consent sheets':'Some else is in charge',
                    'The staff on duty will talk to mothers about prompts and assist in filling the consent forms.':'Some else is in charge',
                    'Deligate it to one of them':'Some else is in charge',
                    'The Matron will ensure the consent forms are secure':'Some else is in charge',
                    'The MCH nurse will take over':'Some else is in charge',
                    'The deputies will be the custodians':'Some else is in charge',
                    'Hands over to a nurse Colleague':'Some else is in charge',
                    'the others nurses help in enrolling mothers':'Some else is in charge',
                    'Mch deputy in-charge':'Some else is in charge',
                    'Staff on duty talks to mothers about PROMPTs helps them fill  and keeps the forms in safe custody':'Some else is in charge',
                    'Nurse at MCH and maternity':'Some else is in charge',
                    'She will be appointing person to take charge.':'Some else is in charge',
                    'The deputies will be the custodian with supervision from Deputy NMS':'Some else is in charge',
                    'The agSCPHN will appoint another to be the custodian':'Some else is in charge',
                    'The nurse who is serving mothers will take over':'Some else is in charge',
                    'He hands over to the contact person':'Some else is in charge',
                    'The nurse on duty will talk to mums , assist in filling  the consent  sheets and keep it safe':'Some else is in charge',
                    'Anc nurses will take over':'Some else is in charge',
                    'The other nurses enrol mothers':'Some else is in charge',
                    'Delegated to standing in nurse':'Some else is in charge',
                    'Maternity incahrge will be responsible.':'Some else is in charge',
                    'MCH nurse will take over':'Some else is in charge',
                    'The other nurse can still acess the file.':'Some else is in charge',
                    'Deligate the duty to the HRIO':'Some else is in charge',
                    'If the nurse is not around or on referral duties the record officer will talk to the mothers and assist in filling  the consent sheets':'Some else is in charge',
                    'The staff on duty will talk to mothers about PROMPTS and assist them to fill the consent forms':'Some else is in charge',
                    'They hand over to a nominated custodian.':'Some else is in charge',
                    'the other staff take over the enrolment of mothers':'Some else is in charge',
                    'She will see to it done by her  work mate':'Some else is in charge',
                    'Other nurses can still acess the file.':'Some else is in charge',
                    'Have met with the other 4 facility staff and educated them on PROMPTS so in absentia of the contact person they can take up the task':'Some else is in charge',
                    'She handsover to a colleague nurse on duty':'Some else is in charge',
                    'The nurse they work with at MCH will be the custodian':'Some else is in charge',
                    'The incharge will make arrangements to ensure the staff on duty will enroll mothers via posters or consent forms':'Some else is in charge',
                    'He hands over to the mch and maternity incharge.':'Some else is in charge',
                    'The nurse on duty.':'Some else is in charge',
                    "The HRIO who has been enrolling mum's will be the one who stores the files":'Some else is in charge',
                    'The nurse maning the station  and will be responsible for the consent sheets and the contact person will coordinate to ensure its safe custody':'Some else is in charge',
                    'Any other nurse can still acess the file and enroll.':'Some else is in charge',
                    'The MCH incharge who the contact person takes charge':'Some else is in charge',
                    "Other Nurses can still enroll the mum's.":'Some else is in charge',
                    'The linda mama agent will talk to the mothers about PROMPTS and assist the mothers to enroll via posters.':'Some else is in charge',
                    'Deligate the duty to Epsos staff':'Some else is in charge',
                    'A nurse in the MCH will support this course.':'Some else is in charge',
                    'The nursing officer of the facility will be incharge.':'Some else is in charge',
                    'The Nurse present will be responsible for the consent sheets.':'Some else is in charge',
                    'Have met with the department and everyone is willing to ensure mothers get enrolled into the system':'Some else is in charge',
                    'Other nurse can still access the file and enroll mums':'Some else is in charge',
                    'The nurse on duty will be responsible':'Some else is in charge',
                    'Deputy facility in-charge':'Some else is in charge',
                    'Deligate the duty to nurse on duty':'Some else is in charge',
                    'NOTHING':'Not sure',
                    'He will deligate it to the other nurse in the facility':'Some else is in charge',
                    'The facility nutritionist who is the additional contact person':'Some else is in charge',
                    'the other staff take over enrolment of mothers':'Some else is in charge',
                    'The staff on duty will be responsible for assisting the Mums fill the consent forms':'Some else is in charge',
                    'Anc  nurse will take over':'Some else is in charge',
                    'The nurse  on duty will  be responsible for the consent sheets, talk to the mums and assist in filling the sheets.':'Some else is in charge',
                    'She will leave the office keys with  the staff on duty to retrieve the consent sheets.\n\nSensitive other staff members about the program and the need to fill the sheets while on duty.':'Some else is in charge',
                    'The incharge will  sensitive all Nurses about the consent forms and task the staff on duty to assist the mothers to fill and sign the sheets.':'Some else is in charge',
                    'Nurse conducting group ANC':'Some else is in charge',
                    'Delegate to the nurse on duty':'Some else is in charge',
                    'Let the nurse on duty be the custodian':'Some else is in charge',
                    'NSM will be incharge':'Some else is in charge',
                    'The staff on duty will be responsible for helping  mother fill the consent  sheets  and keep it  safe.':'Some else is in charge',
                    'The nurse on duty will take over':'Some else is in charge',
                    'He handsover to his colleagues on duty':'Some else is in charge',
                    'The nurse takes over enrolment of mothers':'Some else is in charge',
                    'Anc nurse and matenity nurse will take over':'Some else is in charge',
                    'MCH incharge will take over':'Some else is in charge',
                    'She will handover to her colleague nurse on duty':'Some else is in charge',
                    'Anc will take over':'Some else is in charge',
                    'The other staff help to enroll mothers':'Some else is in charge',
                    'MCH- Has always embraced team work and therefore hand over is made to the next person.\nMaternity- Subcounty RH to support':'Some else is in charge',
                    'The facilitry contact person becomes the custodian':'Some else is in charge',
                    'other nurses can still acess the file and enroll mums':'Some else is in charge',
                    'The mch incharge takes the lead':'Some else is in charge',
                    'The staff on duty takes the responsibility to talk to the mothers about PROMPTs':'Some else is in charge',
                    'They hand over amongst staffs in the department':'Some else is in charge',
                    'The staff on duty will talk to mums about PROMPTs and help mums  consent':'Some else is in charge',
                    'She hands over to her colleague on duty':'Some else is in charge',
                    'other ANC nurse will take over':'Some else is in charge',
                    'The linda mama agent will enroll mothers via consent forms':'Some else is in charge',
                    'The nurse in charge will take over.':'Some else is in charge',
                    'The contact person.':'Some else is in charge',
                    'Matron will follow up':'Some else is in charge',
                    'The facility to appoint other people working in the same departments.':'Some else is in charge',
                    'They have colleagues whom we have sensitized about the program':'Some else is in charge',
                    'He has assigned the MCH and the maternity incharges to take charge of the forms.':'Some else is in charge',
                    'The other nurses take over to cover that':'Some else is in charge',
                    'She will nominate someone to take charge of that.':'Some else is in charge',
                    'It is placed at the nursing station table':'Some else is in charge',
                    'Any Nurse on duty can still access the file.':'Some else is in charge',
                    'other staff are able to access the consent sheets.':'Some else is in charge',
                    'Other Nurse can still access the file':'Some else is in charge',
                    'The file will be stored in Room 9 that deals with New ANC mothers only and if the custodian is not around the person in room 9 that day will be responsible for the file':'Some else is in charge',
                    'They handover the custody to their colleague and inform the PA':'Some else is in charge',
                    'Hands over custody to her colleague and inform the PA':'Some else is in charge',
                    'The nurse incharge will take over assisted by the sphn since his office is within the hospital':'Some else is in charge',
                    'The next staff will be the custodian. (The service providers work on a monthly rotational basis)':'Some else is in charge',
                    'The facility have nominated their assistants to take charge of the sheets while the direct contact persons are away.':'Some else is in charge',
                    'She will handover to her colleague  who is also a mentor mother':'Some else is in charge',
                    'The deputy at MCH will be the custodian':'Some else is in charge',
                    'Leaves the file at the Nurse desk for easy Access to whoever is on duty':'Some else is in charge',
                    '2 more people have been nominated to take charge of this.':'Some else is in charge',
                    'The MCH and Maternity in-charge':'Some else is in charge',
                    'ANC nurses will take over':'Some else is in charge',
                    'The nutritionist will handle enrollments. She is all round and the incharge trusts her work':'Some else is in charge',
                    'Deligate it to the nurse on duty.':'Some else is in charge',
                    'the other nurses take over enroll of mothers':'Some else is in charge',
                    'Might miss out on enrollment since this is currently the only person who can support with enrollment. The facility is heavily understaffed.':'Some else is in charge',
                    'The facility matron will take charge.':'Some else is in charge',
                    'Nurse in-charge':'Some else is in charge',

                     })


print(df_modified['What happens when this person is not around?'].value_counts())

Some else is in charge    1108
                           196
N/A                          6
Not sure                     4
NA                           2
Name: What happens when this person is not around?, dtype: int64


In [ ]:
#Name of the Facility(As it appears on KHIS)'

df_modified['Name of the Facility(As it appears on KHIS)'] = df_modified['Name of the Facility(As it appears on KHIS)'].str.title().str.strip()

print(df_modified['Name of the Facility(As it appears on KHIS)'].value_counts())

                                         196
Kitengela Sub County Hospital              5
Vipingo Rural Demonstration Center         4
Ngong Sub County Hospital                  4
Kilifi County Hospital                     4
                                        ... 
Devine Mercy Aluor Mission Hospital        1
St. Theresa Mission Hospital (Kiirua)      1
Mabusi Health Center                       1
Kiriani Dispensary                         1
Ruiru Sub-County                           1
Name: Name of the Facility(As it appears on KHIS), Length: 815, dtype: int64


In [ ]:
pip install fuzzywuzzy

In [ ]:
pip install transformers

In [ ]:
pip install torch torchvision torchaudio

In [ ]:
from fuzzywuzzy import process
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import torch




# Preprocess clean facility names from df_facilities
clean_facilities = df_Facilities['Facility_Name'].apply(lambda x: x.lower().strip())

# Load the pretrained model and tokenizer
model_name = "bert-base-uncased"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define a function to predict dirty name based on clean name
def predict_dirty_name(clean_name):
    tokenized_input = tokenizer(clean_name, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        output = model(**tokenized_input)
        predicted_output = output.logits.argmax(dim=1).item()
    return clean_name  # Placeholder, replace with actual prediction logic

# Apply predictions and fuzzy string matching to df_modified
for i, row in df_modified.iterrows():
    dirty_name = row['Name of the Facility(As it appears on KHIS)'].lower().strip()
    predicted_name = predict_dirty_name(dirty_name)  # Replace with actual prediction function
    best_match = process.extractOne(predicted_name, clean_facilities)
    if best_match[1] > 80:  # Adjust the similarity threshold as needed
        df_modified.at[i, 'Name of the Facility(As it appears on KHIS)'] = best_match[0]




Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Row 1 processed.
Row 2 processed.
Row 3 processed.
Row 4 processed.
Row 5 processed.
Row 6 processed.
Row 7 processed.
Row 8 processed.
Row 9 processed.
Row 10 processed.
Row 11 processed.
Row 12 processed.
Row 13 processed.
Row 14 processed.
Row 15 processed.
Row 16 processed.
Row 17 processed.


Row 18 processed.
Row 19 processed.
Row 20 processed.
Row 21 processed.
Row 22 processed.
Row 23 processed.
Row 24 processed.
Row 25 processed.
Row 26 processed.
Row 27 processed.
Row 28 processed.
Row 29 processed.
Row 30 processed.
Row 31 processed.
Row 32 processed.


Row 33 processed.
Row 34 processed.
Row 35 processed.
Row 36 processed.
Row 37 processed.
Row 38 processed.
Row 39 processed.
Row 40 processed.
Row 41 processed.
Row 42 processed.
Row 43 processed.
Row 44 processed.
Row 45 processed.
Row 46 processed.
Row 47 processed.
Row 48 processed.
Row 49 processed.


Row 50 processed.
Row 51 processed.
Row 52 processed.
Row 53 processed.
Row 54 processed.
Row 55 processed.
Row 56 processed.
Row 57 processed.
Row 58 processed.
Row 59 processed.
Row 60 processed.
Row 61 processed.
Row 62 processed.
Row 63 processed.
Row 64 processed.
Row 65 processed.
Row 66 processed.
Row 67 processed.

...

Row 133 processed.
Row 134 processed.
Row 135 processed.
Row 136 processed.
Row 137 processed.
Row 138 processed.
Row 139 processed.
Row 140 processed.
Row 141 processed.
Row 142 processed.
Row 143 processed.
Row 144 processed.
Row 145 processed.
Row 146 processed.
Row 147 processed.
Row 148 processed.
Row 149 processed.


Row 150 processed.
Row 151 processed.
Row 152 processed.
Row 153 processed.
Row 154 processed.
Row 155 processed.
Row 156 processed.
Row 157 processed.
Row 158 processed.
Row 159 processed.
Row 160 processed.
Row 161 processed.
Row 162 processed.
Row 163 processed.
Row 164 processed.


Row 165 processed.
Row 166 processed.
Row 167 processed.
Row 168 processed.
Row 169 processed.
Row 170 processed.
Row 171 processed.
Row 172 processed.
Row 173 processed.
Row 174 processed.
Row 175 processed.
Row 176 processed.
Row 177 processed.
Row 178 processed.
Row 179 processed.
Row 180 processed.


Row 181 processed.
Row 182 processed.
Row 183 processed.
Row 184 processed.
Row 185 processed.
Row 186 processed.
Row 187 processed.
Row 188 processed.
Row 189 processed.
Row 190 processed.
Row 191 processed.
Row 192 processed.
Row 193 processed.
Row 194 processed.
Row 195 processed.
Row 196 processed.
Row 197 processed.
Row 198 processed.
Row 199 processed.
Row 200 processed.
Row 201 processed.
Row 202 processed.
Row 203 processed.
Row 204 processed.
Row 205 processed.
Row 206 processed.
Row 207 processed.
Row 208 processed.
Row 209 processed.
Row 210 processed.
Row 211 processed.
Row 212 processed.
Row 213 processed.
Row 214 processed.
Row 215 processed.
Row 216 processed.
Row 217 processed.
Row 218 processed.
Row 219 processed.
Row 220 processed.
Row 221 processed.
Row 222 processed.
Row 223 processed.
Row 224 processed.
Row 225 processed.
Row 226 processed.
Row 227 processed.
Row 228 processed.
Row 229 processed.
Row 230 processed.
Row 231 processed.
Row 232 processed.
Row 233 proc

Row 1315 processed.
Row 1316 processed.


In [ ]:
#'Name of the County',
df_modified = df_modified[df_modified['Name of the County'] != '|']
df_modified['Name of the County'] = df_modified['Name of the County'].str.capitalize().str.strip()

print(df_modified['Name of the County'].value_counts())

Nairobi       136
Kilifi        109
Kakamega      103
Siaya          85
Murang'a       75
Makueni        73
Kitui          73
Meru           68
Bungoma        64
Kisii          60
Kiambu         60
Nakuru         59
Migori         59
Machakos       50
Busia          48
Kericho        45
Kajiado        43
Nyeri          40
Kirinyanga     37
Narok          27
Homabay         1
Name: Name of the County, dtype: int64


In [ ]:
#'Name of the Sub county'

df_modified['Name of the Sub county'] = df_modified['Name of the Sub county'].str.title().str.strip()

df_modified["Name of the Sub county"] = df_modified["Name of the Sub county"].replace({
    'Gem Sub County':'Gem',
    'Gem Sub County Hospital':'Gem',
    'Marhioya':'Mathioya',
    'Malava Sub County':'Malava',
    'Ainamoi Sub county':'Ainamoi',
    'Ainamoi Sub County':'Ainamoi',
    'Ainamoi Health Center':'Ainamoi',
    'Libra/Langata':'Kibra/Langata',
    'Makadara Sub-County':'Makadara',
    'Kangema Sub County':'Kangema',
    'Ugunja Sub County':'Ugunja',
    'Kasarani Sub-County':'Kasarani',
    'Westlands Sub-County':'Westlands',
    'Bureti Sub County':'Bureti',
    'Kipkelion East subcounty':'Kipkelion East',
    'Kipkelion East Sub County':'Kipkelion East',
    'Ruaraka/Roysambu Sub-county':'Ruaraka/Roysambu',
    'Ruaraka/Roysambu Sub-County':'Ruaraka/Roysambu',
    'Kiharu Sub County':'Kiharu',
    'Kamukunji Sub-County':'Kamukunji',
    'Naivasha Sub County':'Naivasha',
    'Naivasha Sub County':'Naivasha',
    'Subukia Sub County':'Subukia',
    'Nakuru East Sub County':'Nakuru East',
    'Nakuru North Sub County':'Nakuru North',
    'Kipkelion East Subcounty':'Kipkelion East'




})

print(df_modified['Name of the Sub county'].value_counts())

                1120
Kangema            9
Kandara            6
Kigumo             5
Malava             5
                ... 
Ruiru              1
Alego Usonga       1
Nakuru North       1
Nakuru East        1
Sigowet/Soin       1
Name: Name of the Sub county, Length: 90, dtype: int64


In [ ]:
# I installed this for string matching since typing all words is time consuming.
#pip install fuzzywuzzy

In [ ]:
# Define the new column name
new_column_name = "What are some of the challenges faced by the facility?(document on the facility supervision book tool)"

# Get the index of the current column name
current_column_index = df_modified.columns.get_loc("What are some of the challenges and  action items agreed with the facility going forward(document on the facility supervision book too)\neg: \nChallenge 1: There is staff shortage.\nAction 1: The staff to utilize the Health talks to inform all mothers about PROMPTS then mums can sign the forms\n")

# Rename the column using the rename function
df_modified.rename(columns={df_modified.columns[current_column_index]: new_column_name}, inplace=True)


In [ ]:
!pip install fuzzywuzzy



In [ ]:
!apt-get -qq install -y python-dev

E: Package 'python-dev' has no installation candidate


In [ ]:
!pip install python-Levenshtein


In [ ]:
from fuzzywuzzy import fuzz
import pandas as pd

# Standardized categories
standardized_categories = ['Staff shortage', 'Financial constraints', 'Inadequate space', 'Any other specify']

def map_response(response):
    if pd.isna(response) or str(response).strip().lower() in ['na', 'n/a']:
        return 'NA'  # Return 'NA' for missing values (NaN) and variations of 'NA'
    for category in standardized_categories:
        # Compare similarity between the response and each standardized category
        similarity = fuzz.partial_ratio(str(response).lower(), category.lower())
        if similarity >= 82:  # You can adjust the similarity threshold as needed
            return category
    return 'Any other specify'  # Return 'Any other specify' for other responses

# Store the exact values in the 'What are some of the challenges faced by the facility?(document on the facility supervision book tool)' column
exact_values = df_modified['What are some of the challenges faced by the facility?(document on the facility supervision book tool)']

# Create the 'Normalized_Response' column with the standardized categories
df_modified['Normalized_Response'] = df_modified['What are some of the challenges faced by the facility?(document on the facility supervision book tool)'].apply(map_response)

# Create new columns for each unique response with Boolean values
for category in standardized_categories:
    df_modified[category] = (df_modified['Normalized_Response'] == category).astype(int)

# Create the "Any other specify" column to hold '1' for any value present but absent in the first 3 categories
df_modified['Any other specify'] = df_modified.apply(lambda row: 1 if row['Normalized_Response'] == 'Any other specify' else 0, axis=1)

# Create the "Other-challenges" column to hold the exact values from the main column, where "Any other specify" is '1' (TRUE)
df_modified['Other-challenges'] = df_modified['What are some of the challenges faced by the facility?(document on the facility supervision book tool)']
df_modified.loc[df_modified['Any other specify'] == 0, 'Other-challenges'] = ''

# Reorder columns to place new columns just after 'What are some of the challenges faced by the facility?(document on the facility supervision book tool)'
index_of_main_column = df_modified.columns.get_loc('What are some of the challenges faced by the facility?(document on the facility supervision book tool)') + 1
for response in standardized_categories + ['Other-challenges']:
    new_column = df_modified.pop(response)
    df_modified.insert(index_of_main_column, response, new_column)
    index_of_main_column += 1

# Drop the 'Normalized_Response' column
df_modified.drop(['Normalized_Response'], axis=1, inplace=True)



In [ ]:
# Define the name of the new column
new_column_name = 'What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)'

# Create the new column and copy data from the existing column
df_modified[new_column_name] = df_modified['What are some of the challenges faced by the facility?(document on the facility supervision book tool)']

# Find the index of 'Other-challenges' column
index_of_other_challenges = df_modified.columns.get_loc('Other-challenges')

# Reorder columns to place the new column just after 'Other-challenges'
cols = df_modified.columns.tolist()
cols.remove(new_column_name)  # Remove the column if it was inadvertently added at the end
cols.insert(index_of_other_challenges + 1, new_column_name)
df_modified = df_modified[cols]


In [ ]:
#'What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)'

from fuzzywuzzy import fuzz

# Standardized categories
standardized_categories = ['The staff to utilize the Health talks to talk about PROMPTs',
                           'The facility to find a secure place for consent sheets',
                           'The facility to allocate another person to take charge of consent sheets and enrollments',
                           'Organizing a meeting to re-sensitize PROMPTs program',
                           'Follow-up with hospital administration']

def map_response(response):
    if pd.isna(response) or str(response).strip().lower() in ['na', 'n/a', 'None']:
        return 'NA'  # Return 'NA' for missing values (NaN) and variations of 'NA'

    # We used fuzzywuzzy for similarity comparison
    max_similarity = 0
    best_match = None
    for category in standardized_categories:
        similarity = fuzz.partial_ratio(str(response).lower(), category.lower())
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = category

    if max_similarity >= 60:  # You can adjust the similarity threshold as needed
        return best_match
    else:
        return response  # Return the original response if no match is found in the standardized categories

# Store the exact values in the 'What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)' column
exact_values = df_modified['What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)']

# Create the 'Normalized_Response' column with the standardized categories
df_modified['Normalized_Response_Action'] = df_modified['What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)'].apply(map_response)

# Create new columns for each unique response with Boolean values
for category in standardized_categories:
    df_modified[category] = (df_modified['Normalized_Response_Action'] == category).astype(int)

# Create the 'Other-action items' column to hold the exact values from the main column
df_modified['Other-action items'] = df_modified['What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)']

# Check if the exact value is not present in any of the standardized categories columns
# If not present, put the exact value, otherwise set it to empty
for index, row in df_modified.iterrows():
    for category in standardized_categories:
        if row[category] == 1:
            # Exact value is present in at least one of the standardized categories columns
            df_modified.at[index, 'Other-action items'] = ''
            break

# Reorder columns to place new columns just after 'What are some of the challenges faced by the facility?(document on the facility supervision book tool)'
index_of_main_column = df_modified.columns.get_loc('What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)') + 1
for response in standardized_categories + ['Other-action items']:
    new_column = df_modified.pop(response)
    df_modified.insert(index_of_main_column, response, new_column)
    index_of_main_column += 1

# Drop the 'Normalized_Response' column
df_modified.drop(['Normalized_Response_Action'], axis=1, inplace=True)


In [ ]:
import re

def extract_percentage(text):
    # Clean the text and remove non-digit characters
    cleaned_text = re.sub(r'[^\d.]', '', str(text))

    # Check if the cleaned text is empty or contains 'N/a', 'NA', or 'None'
    if not cleaned_text or cleaned_text.lower() in ['n/a', 'na', 'none']:
        return ''  # Return empty string for missing values or 'N/a', 'NA', 'None'

    # Find all percentage values in the observation
    percentages = re.findall(r'\d+(\.\d+)?%', cleaned_text)

    # Check if there are any digits in the observation
    digits = re.findall(r'\d+(\.\d+)?', cleaned_text)

    # If there are digits but no percentage sign, treat them as percentages by adding the percentage sign
    if digits and not percentages:
        cleaned_text += '%'
        percentages.append(cleaned_text)

    # Now, we proceed to categorize the percentage values
    if percentages:
        for percentage in percentages:
            # Remove the percentage sign and convert to a float
            percentage = float(percentage.replace('%', ''))
            # Handle outliers (percentages greater than 100%)
            if percentage > 100:
                percentage /= 100  # Convert to a value in the range [0, 1]
            # Categorize the percentage value
            if 50 <= percentage < 70:
                return '50-69%'
            elif 70 <= percentage < 90:
                return '70-89%'
            elif percentage >= 90:
                return '90-100%'

    return ''  # Return empty string for text without percentage values or non-percentage values

# Create the new 'Percentage_Category' column with the categorized values
df_modified['Percentage_Category'] = df_modified['What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts? '].apply(extract_percentage)

# Replace NaN values with empty strings
df_modified['Percentage_Category'].fillna('', inplace=True)


In [ ]:
# the code just confirms if our code is working well
selected_columns_2 = ['What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts? ',
                      'Percentage_Category'
                    ]

# Filter rows where 'Other-action items' column is not empty
condition = df_modified['Percentage_Category'].ne('')
df_modified.loc[condition, selected_columns_2].head(3)


In [ ]:
import re

def clean_numeric_value(text):
    # Strip leading and trailing spaces from the text
    cleaned_text = str(text).strip()

    # Check if the cleaned text is not empty and not equal to '.'.
    if cleaned_text and cleaned_text != '.':
        # If the cleaned text contains 'NA', 'None', 'N/a', etc., return NaN
        if re.search(r'\b(?:NA|None|N/a)\b', cleaned_text, re.I):
            return float('nan')

        # If the cleaned text contains the word '1st', return NaN
        if '1st' in cleaned_text:
            return float('nan')

        # Check if there is a percentage sign in the text
        if '%' in cleaned_text:
            # Find the numeric value that ends with a percentage sign
            numeric_match = re.search(r'\b\d+(\.\d+)?(?=%)\b', cleaned_text)
            if numeric_match:
                numeric_value = numeric_match.group()
                try:
                    return float(numeric_value)
                except ValueError:
                    pass

        # If no percentage sign is found or the extraction failed, try to extract the last numeric part
        numeric_match = re.search(r'\b\d+(\.\d+)?\b', cleaned_text)
        if numeric_match:
            numeric_value = numeric_match.group()
            try:
                return float(numeric_value)
            except ValueError:
                pass

    # Return NaN for empty or invalid numeric values
    return float('nan')

# Clean the 'What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?' column
df_modified['Percentage_Cleaned'] = df_modified['What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts? '].apply(clean_numeric_value)



In [ ]:
# What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?

# Function to categorize the percentage values
def categorize_percentage(value):
    if value < 50:
        return '<50'
    elif 50 <= value < 70:
        return '50-69%'
    elif 70 <= value < 90:
        return '70-89%'
    elif 90 <= value <= 100:
        return '90-100%'
    else:
        return ''

# Create the new 'Percentage_Category' column based on the values from 'Percentage_Cleaned'
df_modified['Percentage_Category'] = df_modified['Percentage_Cleaned'].apply(categorize_percentage)

# Drop the original 'What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?' column
df_modified.drop('What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts? ', axis=1, inplace=True)

# Insert the new 'Percentage_Category' column next to the 'Observe the nurse explain the information to mums, What would limit comprehension of the consent sheets?' column
observation_column_index = df_modified.columns.get_loc('Observe the nurse explain the information to mums, What would limit comprehension of the consent sheets?')
df_modified.insert(observation_column_index + 1, 'What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?_2', df_modified['Percentage_Category'])

# Drop the temporary 'Percentage_Category' column
df_modified.drop('Percentage_Category', axis=1, inplace=True)

# Drop the 'Percentage_Cleaned' column
df_modified.drop('Percentage_Cleaned', axis=1, inplace=True)


In [ ]:
# What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?
import re

def clean_numeric_value(text):
    # Strip leading and trailing spaces from the text
    cleaned_text = str(text).strip()

    # Check if the cleaned text is not empty and not equal to '.'.
    if cleaned_text and cleaned_text != '.':
        # If the cleaned text contains 'NA', 'None', 'N/a', etc., return NaN
        if re.search(r'\b(?:NA|None|N/a)\b', cleaned_text, re.I):
            return float('nan')

        # If the cleaned text contains the word '1st', return NaN
        if '1st' in cleaned_text:
            return float('nan')

        # Check if there is a percentage sign in the text
        if '%' in cleaned_text:
            # Find the numeric value that ends with a percentage sign
            numeric_match = re.search(r'\b\d+(\.\d+)?(?=%)\b', cleaned_text)
            if numeric_match:
                numeric_value = numeric_match.group()
                try:
                    return float(numeric_value)
                except ValueError:
                    pass

        # If no percentage sign is found or the extraction failed, try to extract the last numeric part
        numeric_match = re.search(r'\b\d+(\.\d+)?\b', cleaned_text)
        if numeric_match:
            numeric_value = numeric_match.group()
            try:
                return float(numeric_value)
            except ValueError:
                pass

    # Return NaN for empty or invalid numeric values
    return float('nan')

# Clean the 'What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?' column
df_modified['Percentage_Cleaned_2'] = df_modified['What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?'].apply(clean_numeric_value)



In [ ]:
# What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?

# Function to categorize the percentage values
def categorize_percentage(value):
    if value < 50:
        return '<50'
    elif 50 <= value < 70:
        return '50-69%'
    elif 70 <= value < 90:
        return '70-89%'
    elif 90 <= value <= 100:
        return '90-100%'
    else:
        return ''

# Create the new 'Percentage_Category' column based on the values from 'Percentage_Cleaned'
df_modified['Percentage_Category_2'] = df_modified['Percentage_Cleaned_2'].apply(categorize_percentage)

# Drop the original 'What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?' column
df_modified.drop('What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?', axis=1, inplace=True)

# Insert the new 'Percentage_Category' column next to the 'Observe the nurse explain the information to mums, What would limit comprehension of the consent sheets?' column
observation_column_index = df_modified.columns.get_loc('What percentage of ANC clients, that visited the facility in the previous month, signed consent sheets for enrollment to prompts?_2')
df_modified.insert(observation_column_index + 1, 'What percentage of mums, that delivered at the facility in the previous month, signed consent sheets for enrollment to prompts?_2', df_modified['Percentage_Category_2'])

# Drop the temporary 'Percentage_Category' column
df_modified.drop('Percentage_Category_2', axis=1, inplace=True)

# Drop the 'Percentage_Cleaned' column
df_modified.drop('Percentage_Cleaned_2', axis=1, inplace=True)


In [ ]:
#'If yes, which facility changes or improvements have been made?'

from fuzzywuzzy import fuzz

# Standardized categories
standardized_categories = ['Infrastructure and equipment','Human resource',
                           'Supplies and commodities','Training and capacity building',
                           'Health information','Health financing']

def map_response(response):
    if pd.isna(response) or str(response).strip().lower() in ['na', 'n/a', 'None']:
        return 'NA'  # Return 'NA' for missing values (NaN) and variations of 'NA'

    # We used fuzzywuzzy for similarity comparison
    max_similarity = 0
    best_match = None
    for category in standardized_categories:
        similarity = fuzz.partial_ratio(str(response).lower(), category.lower())
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = category

    if max_similarity >= 60:  # You can adjust the similarity threshold as needed
        return best_match
    else:
        return response  # Return the original response if no match is found in the standardized categories

# Store the exact values in the 'What are some of the action items agreed with the facility going forward? (document on the facility supervision book tool)' column
exact_values = df_modified['If yes, which facility changes or improvements have been made?']

# Create the 'Normalized_Response' column with the standardized categories
df_modified['Normalized_Response_Action'] = df_modified['If yes, which facility changes or improvements have been made?'].apply(map_response)

# Create new columns for each unique response with Boolean values
for category in standardized_categories:
    df_modified[category] = (df_modified['Normalized_Response_Action'] == category).astype(int)

# Create the 'Any_Other_Improvement_Specify' column to hold the exact values from the main column
df_modified['Any_Other_Improvement_Specify'] = df_modified['If yes, which facility changes or improvements have been made?']

# Check if the exact value is not present in any of the standardized categories columns
# If not present, put the exact value, otherwise set it to empty
for index, row in df_modified.iterrows():
    for category in standardized_categories:
        if row[category] == 1:
            # Exact value is present in at least one of the standardized categories columns
            df_modified.at[index, 'Any_Other_Improvement_Specify'] = ''
            break

# Reorder columns to place new columns just after 'If yes, which facility changes or improvements have been made?'
index_of_main_column = df_modified.columns.get_loc('If yes, which facility changes or improvements have been made?') + 1
for response in standardized_categories + ['Any_Other_Improvement_Specify']:
    new_column = df_modified.pop(response)
    df_modified.insert(index_of_main_column, response, new_column)
    index_of_main_column += 1

# Drop the 'Normalized_Response' column
df_modified.drop(['Normalized_Response_Action'], axis=1, inplace=True)


In [ ]:
#'If you answered No above, please provide a reason why.'

from fuzzywuzzy import fuzz

# Standardized categories
standardized_categories = ['Consent sheets depleted',
                           'Consent sheets misplaced']

def map_response(response):
    if pd.isna(response) or str(response).strip().lower() in ['na', 'n/a', 'None']:
        return 'NA'  # Return 'NA' for missing values (NaN) and variations of 'NA'

    # We used fuzzywuzzy for similarity comparison
    max_similarity = 0
    best_match = None
    for category in standardized_categories:
        similarity = fuzz.partial_ratio(str(response).lower(), category.lower())
        if similarity > max_similarity:
            max_similarity = similarity
            best_match = category

    if max_similarity >= 80:  # You can adjust the similarity threshold as needed
        return best_match
    else:
        return response  # Return the original response if no match is found in the standardized categories

# Store the exact values in the 'If you answered No above, please provide a reason why.' column
exact_values = df_modified['If you answered No above, please provide a reason why.']

# Create the 'Normalized_Response' column with the standardized categories
df_modified['Normalized_Response_Action'] = df_modified['If you answered No above, please provide a reason why.'].apply(map_response)

# Create new columns for each unique response with Boolean values
for category in standardized_categories:
    df_modified[category] = (df_modified['Normalized_Response_Action'] == category).astype(int)

# Create the 'Any_Other_on_consent_availabilty_Specify' column to hold the exact values from the main column
df_modified['Any_Other_on_consent_availabilty_Specify'] = df_modified['If you answered No above, please provide a reason why.']

# Check if the exact value is not present in any of the standardized categories columns
# If not present, put the exact value, otherwise set it to empty
for index, row in df_modified.iterrows():
    for category in standardized_categories:
        if row[category] == 1:
            # Exact value is present in at least one of the standardized categories columns
            df_modified.at[index, 'Any_Other_on_consent_availabilty_Specify'] = ''
            break

# Reorder columns to place new columns just after 'If yes, which facility changes or improvements have been made?'
index_of_main_column = df_modified.columns.get_loc('If you answered No above, please provide a reason why.') + 1
for response in standardized_categories + ['Any_Other_on_consent_availabilty_Specify']:
    new_column = df_modified.pop(response)
    df_modified.insert(index_of_main_column, response, new_column)
    index_of_main_column += 1

# Drop the 'Normalized_Response' column
df_modified.drop(['Normalized_Response_Action'], axis=1, inplace=True)


In [ ]:


# This block of codes reads the cleaned data to a new worksheet in the same workbook in our drive

# Add the new worksheet and update data to it
def add_data_to_worksheet(dataframe, workbook_name, existing_worksheet_name, new_worksheet_name, default_num_rows=100, default_num_cols=20):
    # Get the shape of the DataFrame (number of rows and columns)
    num_rows, num_cols = dataframe.shape

    # Determine the number of rows and columns for the new worksheet
    rows = num_rows if num_rows > default_num_rows else default_num_rows
    cols = num_cols if num_cols > default_num_cols else default_num_cols

    # Open the workbook and existing worksheet
    workbook = gc.open(workbook_name)
    worksheet = workbook.worksheet(existing_worksheet_name)

    # Add the new worksheet with the determined number of rows and columns
    new_worksheet = workbook.add_worksheet(title=new_worksheet_name, rows=rows, cols=cols)

    # Convert DataFrame data to a list of lists with all float values formatted as strings
    data_to_insert = dataframe.astype(str).values.tolist()

    # Insert header row and data to the new worksheet
    new_worksheet.update('A1', [dataframe.columns.tolist()] + data_to_insert)

# Example usage
# Assuming you have your DataFrame named df_modified
add_data_to_worksheet(df_modified, workbook_name='PROMPTS_Dat', existing_worksheet_name='PROMPTS', new_worksheet_name='Clean_Data', default_num_rows=100, default_num_cols=20)


APIError: ignored